# Фичи по диалогам   

Цель: предсказать для каждого пользователя взятие/ невзятие каждого из четырех продуктов **в течение месяца после отчетной даты**, исторические данные по ним находятся в targets

In [1]:
import numpy as np

import pandas as pd
from pandas.api.types import is_float_dtype, is_integer_dtype

from collections import Counter
from sklearn.utils import resample

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

import math

from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity

import gc
import glob
import pyarrow.parquet as pq
from tqdm import trange, tqdm

C:\Users\Profit77\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.decomposition import PCA
from scipy.spatial.distance import pdist, squareform

In [3]:
from typing import List, Optional, Tuple

In [4]:
import warnings

warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=UserWarning, module='pandas')

In [5]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
# найтройки
# Убираем ограничение отображемых колонок
pd.set_option("display.max_columns", None)
# Устанавливаем тему по умолчанию
sb_dark = sns.dark_palette('skyblue', 8, reverse=True) # teal
sns.set(palette=sb_dark)

In [6]:
# Включаем tqdm для pandas, чтобы можно было запускать progress_apply() вместо простого apply()
tqdm.pandas() 
pd.options.display.max_columns = None
pd.options.display.max_rows = 200

In [7]:
eps = 1e-6

In [12]:
PATH = ''
PATH_DATASET = PATH + 'datasets/sber_source/'
PATH_DATASET_OUTPUT = PATH + 'datasets/'

PATH_DATASET_DIALOG_TRAIN = PATH_DATASET + 'dial_train.parquet/'
PATH_DATASET_DIALOG_TEST = PATH_DATASET + 'dial_test.parquet/'

PATH_DATASET_TARGET_TRAIN = PATH_DATASET + 'train_target.parquet/'
PATH_DATASET_TARGET_TEST = PATH_DATASET + 'test_target_b.parquet/'


# dialogs
train_dialog_files = glob.glob(PATH_DATASET_DIALOG_TRAIN + '/*.parquet')
test_dialog_files = glob.glob(PATH_DATASET_DIALOG_TEST + '/*.parquet')

# Файлы таргеты
train_target_files = glob.glob(PATH_DATASET_TARGET_TRAIN + '/*.parquet')
test_target_files = glob.glob(PATH_DATASET_TARGET_TEST + '/*.parquet')


len(train_dialog_files), len(test_dialog_files)

(18, 3)

In [9]:
# Загрузка списка файлов (типа паркет) в один датафрейм
def load_df_by_files(files:list[str]) -> pd.DataFrame:
    union_df = pd.DataFrame()
    for file in tqdm(files):
        current_df = pq.read_table(file).to_pandas()    
        union_df = pd.concat([union_df, current_df])
    return union_df

In [51]:
%%time
# Загружаем диалоги клиентов
# train_dialog_df = load_df_by_files(train_dialog_files)
# test_dialog_df = load_df_by_files(test_dialog_files)
# train_dialog_df.shape, test_dialog_df.shape
# all_dialogs_df = pd.concat([train_dialog_df, test_dialog_df])
all_dialogs_df = load_df_by_files(train_dialog_files + test_dialog_files)
all_dialogs_df.shape

100%|██████████| 21/21 [01:02<00:00,  2.96s/it]

CPU times: total: 30.5 s
Wall time: 1min 2s


(1501735, 3)

In [13]:
%%time
# Загружаем все таргеты
all_target_df = load_df_by_files(train_target_files + test_target_files)
all_target_df.shape

100%|██████████| 22/22 [00:05<00:00,  4.22it/s]

CPU times: total: 5.25 s
Wall time: 5.22 s


(11686066, 6)

In [26]:
# all_dialogs_df['client_id']
# all_dialogs_df[['client_id']].drop_duplicates().to_csv('list_uniq_client_dialog.csv', index=False)

# all_dialogs_df = all_dialogs_df.drop(columns=['date', 'begin_month'])

In [53]:
%%time
from pandas.tseries.offsets import MonthEnd
all_dialogs_df['end_month'] = pd.to_datetime(all_dialogs_df['event_time'], format="%Y%m") + MonthEnd(1)
all_dialogs_df['end_month'] = all_dialogs_df['end_month'].dt.date
all_dialogs_df['report_next_end'] = all_dialogs_df['end_month'] + relativedelta(months=1)
all_dialogs_df.shape

CPU times: total: 6.44 s
Wall time: 7.62 s


(1501735, 5)

In [54]:
# all_dialogs_df = all_dialogs_df[:1000]
all_dialogs_df = all_dialogs_df.drop(columns=['event_time', 'end_month'], errors='ignore')
all_dialogs_df['report_next_end'] = pd.to_datetime(all_dialogs_df['report_next_end'])
all_dialogs_df = all_dialogs_df.set_index(['client_id', 'report_next_end'])
all_dialogs_df.shape

(1501735, 1)

In [29]:
# # end_date = start_date + relativedelta(months=1) - relativedelta(days=1)
# # report_next_end
# all_dialogs_df['end_month'] + relativedelta(months=1)

# Рассчитываем хорошие и плохие диалоги

In [47]:
all_target_df['mon'] = pd.to_datetime(all_target_df['mon'])
all_target_df = all_target_df.rename(columns={'mon': 'report_next_end'})
all_target_df = all_target_df.set_index(['client_id', 'report_next_end'])
all_target_df['is_target'] = all_target_df[['target_1', 'target_2', 'target_3', 'target_4']].max(axis=1)
all_target_df

target_1  \
client_id                                          report_next_end             
1d4ebf30ab5b981c60a6596e8daa4bdc0646163e50e4861... 2022-06-30              0   
                                                   2022-07-31              0   
                                                   2022-08-31              0   
                                                   2022-09-30              0   
                                                   2022-10-31              0   
...                                                                      ...   
10720d45fe5c441e85eb7ef5271e620be56edb2de87dc29... 2022-10-31              0   
c879fae5376c00b5d56098cfe450c755330e44a351eaa4b... 2022-07-31              0   
771c72f26c0036a4fa6d5e965628e32efa94ed141033dbc... 2022-11-30              0   
c3f8a8f4ff091e711ac102c77f46ec309b5ec27b3a0ed35... 2022-10-31              0   
c1303cfaeb348871e53ecdb5113a9837da23bd60af47638... 2022-05-31              0   

                                                                    target_2  \
client_id                                          report_next_end             
1d4ebf30ab5b981c60a6596e8daa4bdc0646163e50e4861... 2022-06-30              0   
                                                   2022-07-31              0   
                                                   2022-08-31              0   
                                                   2022-09-30              0   
                                                   2022-10-31              0   
...                                                                      ...   
10720d45fe5c441e85eb7ef5271e620be56edb2de87dc29... 2022-10-31              0   
c879fae5376c00b5d56098cfe450c755330e44a351eaa4b... 2022-07-31              0   
771c72f26c0036a4fa6d5e965628e32efa94ed141033dbc... 2022-11-30              0   
c3f8a8f4ff091e711ac102c77f46ec309b5ec27b3a0ed35... 2022-10-31              0   
c1303cfaeb348871e53ecdb5113a9837da23bd60af47638... 2022-05-31              0   

                                                                    target_3  \
client_id                                          report_next_end             
1d4ebf30ab5b981c60a6596e8daa4bdc0646163e50e4861... 2022-06-30              0   
                                                   2022-07-31              0   
                                                   2022-08-31              0   
                                                   2022-09-30              0   
                                                   2022-10-31              0   
...                                                                      ...   
10720d45fe5c441e85eb7ef5271e620be56edb2de87dc29... 2022-10-31              0   
c879fae5376c00b5d56098cfe450c755330e44a351eaa4b... 2022-07-31              0   
771c72f26c0036a4fa6d5e965628e32efa94ed141033dbc... 2022-11-30              0   
c3f8a8f4ff091e711ac102c77f46ec309b5ec27b3a0ed35... 2022-10-31              0   
c1303cfaeb348871e53ecdb5113a9837da23bd60af47638... 2022-05-31              0   

                                                                    target_4  \
client_id                                          report_next_end             
1d4ebf30ab5b981c60a6596e8daa4bdc0646163e50e4861... 2022-06-30              0   
                                                   2022-07-31              0   
                                                   2022-08-31              0   
                                                   2022-09-30              0   
                                                   2022-10-31              0   
...                                                                      ...   
10720d45fe5c441e85eb7ef5271e620be56edb2de87dc29... 2022-10-31              0   
c879fae5376c00b5d56098cfe450c755330e44a351eaa4b... 2022-07-31              0   
771c72f26c0036a4fa6d5e965628e32efa94ed141033dbc... 2022-11-30              0   
c3f8a8f4ff091e711ac102c77f46ec309b5ec27b3a0ed35... 2022-10-31    

In [60]:
dialogs_target_df = all_target_df.merge(all_dialogs_df, left_index=True, right_index=True)
dialogs_target_df.shape

(707519, 6)

In [99]:
%%time
# Считаем "хорошие" и "плохие" эмбеддинги
target_columns = ['target_1', 'target_2', 'target_3', 'target_4', 'is_target']

target_embeddings_dict ={}
dialog_embeddings = np.vstack(dialogs_target_df['embedding'].apply(np.array).values)

for trg_col in target_columns:
    print(trg_col)
    
    good_sub_dlg_trg_df = dialogs_target_df[dialogs_target_df[trg_col] == 1]
    bad_sub_dlg_trg_df = dialogs_target_df[dialogs_target_df[trg_col] == 0]
    good_embedding = good_sub_dlg_trg_df['embedding'].mean()
    bad_embedding = bad_sub_dlg_trg_df['embedding'].mean()
    
    target_embeddings_dict[trg_col] = {}
    target_embeddings_dict[trg_col]['good'] = good_embedding
    target_embeddings_dict[trg_col]['bad'] = bad_embedding
    
    # Производим расчет косинусного расстояния
    dialogs_target_df[f'cos_sim_{trg_col}_good'] = cosine_similarity(dialog_embeddings, good_embedding[None])
    dialogs_target_df[f'cos_sim_{trg_col}_bad'] = cosine_similarity(dialog_embeddings, bad_embedding[None])

target_1
target_2
target_3
target_4
is_target
CPU times: total: 33.8 s
Wall time: 35.9 s


In [110]:
# Расчет статистик на данных
good_columns = []
bad_columns = []
for trg_col in target_columns:
    print(trg_col)
    good_col = f'cos_sim_{trg_col}_good'
    bad_col = f'cos_sim_{trg_col}_bad'
    good_columns.append(good_col)
    bad_columns.append(bad_col)
    
    dialogs_target_df['dist_bad_good'] = abs(dialogs_target_df[bad_col] - dialogs_target_df[good_col])
    dialogs_target_df['diff_bad_good'] = dialogs_target_df[bad_col] - dialogs_target_df[good_col]
    dialogs_target_df['rel_bad_good'] = dialogs_target_df[bad_col] / (dialogs_target_df[good_col] + eps)

target_1
target_2
target_3
target_4
is_target


In [114]:
for t in ['bad', 'good']:
    dialogs_target_df[f'mean_{t}_cossim'] = dialogs_target_df[bad_columns].mean(axis=1)
    dialogs_target_df[f'max_{t}_cossim'] = dialogs_target_df[bad_columns].max(axis=1)
    dialogs_target_df[f'min_{t}_cossim'] = dialogs_target_df[bad_columns].min(axis=1)
    dialogs_target_df[f'sum_{t}_cossim'] = dialogs_target_df[bad_columns].sum(axis=1)

In [116]:
dialogs_target_df = dialogs_target_df.drop(columns=['target_1', 'target_2', 'target_3', 'target_4', 'is_target', 'embedding'])
dialogs_target_df.shape

(707519, 21)

In [117]:
dialogs_target_df

cos_sim_target_1_good  \
client_id                                          report_next_end                          
1d817e82e1cc594e32e10217b716faa3ad079261975097d... 2023-01-31                    0.947542   
                                                   2023-01-31                    0.966281   
                                                   2023-01-31                    0.976196   
                                                   2023-01-31                    0.968635   
1daf5849cf68892983c584c8ee97b8329307584a834e795... 2022-08-31                    0.869208   
...                                                                                   ...   
9b0d6970ae267b8b45132079161e4232b777c8691730629... 2022-09-30                   -0.363600   
                                                   2022-08-31                    0.303318   
                                                   2022-08-31                   -0.315407   
                                                   2022-08-31                    0.956092   
                                                   2022-08-31                   -0.073392   

                                                                    cos_sim_target_1_bad  \
client_id                                          report_next_end                         
1d817e82e1cc594e32e10217b716faa3ad079261975097d... 2023-01-31                   0.954643   
                                                   2023-01-31                   0.971030   
                                                   2023-01-31                   0.980045   
                                                   2023-01-31                   0.972970   
1daf5849cf68892983c584c8ee97b8329307584a834e795... 2022-08-31                   0.860324   
...                                                                                  ...   
9b0d6970ae267b8b45132079161e4232b777c8691730629... 2022-09-30                  -0.383914   
                                                   2022-08-31                   0.282263   
                                                   2022-08-31                  -0.337657   
                                                   2022-08-31                   0.962331   
                                                   2022-08-31                  -0.096874   

                                                                    cos_sim_target_2_good  \
client_id                                          report_next_end                          
1d817e82e1cc594e32e10217b716faa3ad079261975097d... 2023-01-31                    0.954589   
                                                   2023-01-31                    0.971095   
                                                   2023-01-31                    0.980073   
                                                   2023-01-31                    0.973091   
1daf5849cf68892983c584c8ee97b8329307584a834e795... 2022-08-31                    0.860462   
...                                                                                   ...   
9b0d6970ae267b8b45132079161e4232b777c8691730629... 2022-09-30                   -0.384596   
                                                   2022-08-31                    0.282156   
                                                   2022-08-31                   -0.338056   
                                                   2022-08-31                    0.962230   
                                                   2022-08-31                   -0.097325   

                                                                    cos_sim_target_2_bad  \
client_id                                          report_next_end                         
1d817e82e1cc594e32e10217b716faa3ad079261975097d... 2023-01-31                   0.954423   
                                                   2023-01-31                   0.970886   
                                                   2023-01-31                   0.9

In [118]:
%%time
# Сохраняем статистику по косинусной близости
# dialogs_target_df.to_parquet(PATH_DATASET_OUTPUT + 'good_bad_dialogs_target_df_16_06_2024.parquet')

CPU times: total: 1.41 s
Wall time: 2 s


### Загружаем языковую модель

In [195]:
%%time
# bertmodel = SentenceTransformer('DeepPavlov/mbart-large-50-ru-persona-chat') # 1024 emb
bertmodel = SentenceTransformer('DeepPavlov/bart-base-en-persona-chat') # 768
# bertmodel = SentenceTransformer('DeepPavlov/rubert-base-cased')
# bertmodel = SentenceTransformer('DeepPavlov/t5-wikidata5M-with-neighbors') # 512
# bertmodel = SentenceTransformer('google/bert_uncased_L-12_H-768_A-12') # 768

No sentence-transformers model found with name DeepPavlov/bart-base-en-persona-chat. Creating a new one with MEAN pooling.


CPU times: total: 2.06 s
Wall time: 2.94 s


## Формируем примеры диалогов по тематикам 
На базе https://claude.ai/  


In [259]:
vocab_dialogs = {"Текущие (расчетные) счета": [
    {"Позитивный диалог:": "Клиент: Здравствуйте, я бы хотел открыть расчетный счет в вашем банке для своего бизнеса. Оператор: Хорошо, для этого вам необходимо предоставить следующие документы: свидетельство о регистрации юридического лица, устав, приказ о назначении директора и другие учредительные документы. После их рассмотрения мы сможем открыть вам расчетный счет. Клиент: Отлично, большое спасибо за подробную информацию. Я соберу все необходимые документы и приду в ближайшее отделение."},
    {"Негативный диалог:": "Клиент: Я уже несколько раз обращался с просьбой открыть расчетный счет, но каждый раз мне говорят о новых требованиях к документам! Это издевательство! Оператор: Извините, но мы обязаны соблюдать требования законодательства и внутренних нормативов банка. Возможно, есть какие-то нюансы в вашей ситуации, и поэтому запрашиваются дополнительные документы. Давайте разберемся подробнее, что именно вам требуется?",},
],
"Депозиты (вклады)": [
    {"Позитивный диалог:": "Клиент: Добрый день, какие виды вкладов сейчас наиболее выгодные в вашем банке? Оператор: Здравствуйте. Сейчас наиболее привлекательными являются наши накопительные вклады со ставкой до 7% годовых. Вы можете открыть его с минимальной суммой от 50 000 рублей. Также есть возможность частичного снятия средств без потери процентов.Клиент: Отлично, тогда я бы хотел открыть такой вклад. Скажите, пожалуйста, какие документы нужно принести в отделение?"},
    {"Негативный диалог:": "Клиент: Я открыл вклад в вашем банке 3 месяца назад под 9% годовых. А сегодня вижу, что ставки снизились до 7%. Почему мне не предложили более выгодные условия?Оператор: К сожалению, условия по вкладам фиксируются на момент их открытия и действуют до окончания срока. Изменить ставку на более высокую в течение срока вклада невозможно.Клиент: Это некрасиво по отношению к постоянным клиентам банка! Вы должны ценить нас и предлагать лучшие условия!"},
],
"Кредиты": [
    {"Позитивный диалог:": "Клиент: Здравствуйте, я недавно обращался за кредитом в ваш банк. Могу я узнать, на каком этапе рассмотрения моя заявка?Оператор: Да, конечно. Позвольте уточнить ваши персональные данные... Хорошо, ваша заявка уже одобрена. В ближайшие 2 дня с вами свяжется наш менеджер для дальнейшего оформления кредитной документации.Клиент: Отлично, большое спасибо за оперативность. Буду ждать звонка менеджера."},
    {"Негативный диалог:": "Клиент: Я крайне разочарован работой вашего банка! Мне отказали в кредите, хотя все требования были соблюдены. При этом никаких внятных объяснений не предоставили.Оператор: Сожалею, что у вас сложилось негативное впечатление. Решение об одобрении или отказе выносится на основании тщательного анализа кредитоспособности клиента и опирается на ряд факторов. Я проверю вашу ситуацию и дам более развернутый ответ."},
],
"Кредитные карты": [
    {"Позитивный диалог:": "Клиент: Добрый день, я бы хотел оформить кредитную карту. Какие условия сейчас действуют? Оператор: Здравствуйте. У нас сейчас очень выгодное предложение - кредитная карта с льготным периодом 100 дней без процентов на все покупки. Также есть начисление баллов за покупки, которые можно обменивать на подарки. Клиент: Отличное предложение! Скажите, какие документы нужно предоставить для оформления? Оператор: Для получения карты понадобятся паспорт, справка о доходах и анкета, которую можно заполнить у нас в отделении."},
    {"Негативный диалог:": "Клиент: Добрый день, моя кредитная карта вашего банка заблокирована уже несколько дней без объяснения причин! Требую разъяснить ситуацию! Оператор: Сожалею, ваша кредитная карта действительно была временно заблокирована нашей службой безопасности по причине подозрительной операции. Мы проводили дополнительную проверку, чтобы убедиться, что карта не была использована третьими лицами. Клиент: Это недопустимо! У меня были запланированы важные покупки, а из-за ваших необоснованных действий я понес убытки!"},
],
"Дебетовые карты": [
    {"Позитивный диалог:": "Клиент: Здравствуйте, я хотел бы открыть вклад и оформить к нему дебетовую карту. Оператор: Хорошо, конечно. Вы можете выбрать подходящий вклад на нашем сайте в разделе 'Вклады и инвестиции'. Затем, открыв вклад, вы сможете сразу же оформить и получить дебетовую карту к нему для удобного управления средствами. Клиент: Отлично, спасибо за информацию. Я изучу предложения и открою вклад в ближайшее время."},
    {"Негативный диалог:": "Клиент: Добрый день! Моя зарплатная карта вашего банка была украдена, и с нее сняли все деньги. Почему средства не заблокированы своевременно? Оператор: Сожалею, что такая неприятная ситуация произошла. Для блокировки карты вам необходимо было незамедлительно обратиться в банк после кражи. Без вашего оповещения мы не могли знать о несанкционированных действиях с картой. Клиент: Но служба поддержки совершенно недоступна по вечерам и в выходные! Как клиент мог оперативно"},
],
"Денежные переводы и платежи (внутренние и международные)": [
    {"Позитивный диалог:": "Клиент: Здравствуйте, мне нужно срочно перевести деньги родственникам за границу. Подскажите, как это можно сделать? Оператор: Здравствуйте. Для отправки денежного перевода за рубеж вам необходимо посетить любое отделение нашего банка и оформить международный перевод. Понадобится указать получателя, его реквизиты, сумму и валюту перевода. Комиссия будет зависеть от суммы и выбранной системы перевода. Клиент: Хорошо, спасибо большое за информацию. Тогда я приеду в ближайшее отделение и оформлю перевод."},
    {"Негативный диалог:": "Клиент: Я делал платеж через ваш интернет-банкинг на прошлой неделе, но деньги до сих пор не дошли получателю! Что за задержки? Оператор: Сожалею за доставленные неудобства. Для выяснения ситуации мне необходимо уточнить реквизиты платежа и способ его отправки. Переводы внутри страны обычно проходят в течение 1-2 рабочих дней. Клиент: Это просто безобразие! Я должен был оплатить важную услугу вовремя. Требую немедленного зачисления средств получателю и компенсации за задержку!"},
],
"Инвестиционные продукты (брокерские услуги, ПИФы, доверительное управление)": [
    {"Позитивный диалог:": "Клиент: Добрый день! Меня заинтересовали ваши инвестиционные услуги. Не могли бы вы рассказать подробнее о паевых инвестиционных фондах? Оператор: Конечно. ПИФы - это очень удобный инструмент для вложения средств под управлением профессиональной команды. У нас есть разные типы ПИФов: акций, облигаций, смешанных и других стратегий на выбор в зависимости от ваших инвестиционных целей и предпочтений по риску. Минимальная сумма покупки пая от 10 000 руб. Клиент: Хорошо, спасибо. Я изучу линейку ваших ПИФов и приму решение о покупке пая в ближайшее время."},
    {"Негативный диалог:": "Клиент: Я доверил вашим специалистам управление своим портфелем акций, а он принес убытки за год! Верну ли я свои вложенные средства? Оператор: Сожалею, что сложилась такая ситуация. Ввиду волатильности на рынках невозможно гарантировать прибыльность инвестиционного портфеля в краткосрочной перспективе. Но наши управляющие придерживаются разработанной стратегии, чтобы минимизировать риски. Клиент: Это неприемлемо! Я ожидал защиты моих средств, а не таких потерь. Прошу вернуть мои вложения в полном объеме!"},
],
"Банковские гарантии и аккредитивы": [
    {"Позитивный диалог:": "Клиент: Здравствуйте, мне необходимо получить банковскую гарантию для обеспечения контракта. Каковы условия предоставления гарантий в вашем банке? Оператор: Для оформления гарантии нам потребуются учредительные документы вашей организации, финансовая отчетность, информация о контракте и обеспечении. После рассмотрения документов мы сможем назвать точные условия: сумму, сроки, тарифы. Клиент: Хорошо, большое спасибо. Я соберу пакет документов и приеду в ближайшее отделение для дальнейшего оформления."},
    {"Негативный диалог:": "Клиент: Вы уже месяц рассматриваете нашу заявку на получение аккредитива для расчетов с поставщиком. Сроки контракта горят! Почему так долго? Оператор: Извините за причиненные неудобства. Процедура рассмотрения аккредитивов более длительная, так как требует глубокого анализа контрактов и предполагаемых сделок на предмет экономических рисков. Мы стараемся максимально ускориться. Клиент: Это недопустимая волокита! Из-за ваших проволочек мы можем понести репутационные и финансовые потери. Прошу принять наконец положительное решение по нашему аккредитиву!"},
],
"Услуги инкассации и хранения ценностей": [
    {"Позитивный диалог:": "Клиент: Добрый день! Наша компания хотела бы воспользоваться услугами инкассации для перевозки наличных средств из торговых точек. Расскажите, пожалуйста, как мы можем это организовать? Оператор: Хорошо, для организации инкассаторских услуг вам необходимо заключить с нами договор. Сначала нужно предоставить пакет документов о компании, указать адреса забора и сумки. После этого мы составим маршрут и графики инкассации. Клиент: Отлично, большое спасибо за информацию. В ближайшее время наш специалист свяжется с вами для оформления договора."},
    {"Негативный диалог:": "Клиент: Добрый день! Я крайне возмущен работой вашей инкассаторской службы. Уже второй раз за месяц происходит задержка при вывозе наличных из наших магазинов! Оператор: Сожалею за доставленные неудобства. Такие ситуации недопустимы. Прошу прощения за сложившуюся ситуацию. Обязательно разберемся в причинах задержек и примем необходимые меры. Клиент: Это полное безобразие! Из-за подобных простоев мы несем финансовые потери. Прошу принять жесткие меры к исполнителям и гарантировать своевременное обслуживание по договору!"},
],
"Зарплатные проекты для организаций": [
    {"Позитивный диалог:": "Клиент: Здравствуйте, меня интересует возможность перевода сотрудников нашей компании на зарплатный проект с вашим банком. Не могли бы вы рассказать об условиях и преимуществах? Оператор: Да, с удовольствием. Наш зарплатный проект предлагает ряд выгод как для вашей организации, так и для сотрудников. Для компании - это упрощение процедуры выплат зарплат, снижение издержек на обслуживание и специальные тарифы на РКО. Для сотрудников - бесплатное открытие и обслуживание карт, повышенный кэшбэк, льготные ставки по кредитам. Клиент: Звучит заманчиво. Расскажите, как можно подключиться к зарплатному проекту? Оператор: Для этого необходимо заключить договор зарплатного проекта с банком. Вам понадобится предоставить пакет учредительных документов компании, и мы рассмотрим заявку в короткие сроки."},
    {"Негативный диалог:": "Клиент: Я крайне недоволен вашим зарплатным обслуживанием! Уже третий месяц наши сотрудники жалуются на задержки зарплатных выплат. Это недопустимо! Оператор: Приношу свои извинения за доставленные неудобства. Это недопустимая ситуация, я обязательно разберусь в причинах задержек и передам информацию руководству для принятия мер. Клиент: Ваши извинения ничего не стоят! Из-за ваших постоянных сбоев мы рискуем потерять ценные кадры. Если ситуация не изменится, мы будем вынуждены отказаться от ваших услуг!"},
],
"Приложение банка": [
    {"Позитивный диалог:": "Клиент: Добрый день! Хотел узнать, есть ли у вашего банка мобильное приложение для управления счетами и картами? Оператор: Да, конечно. Мы разработали удобное мобильное приложение, которое позволяет полностью управлять вашими финансами удаленно. В приложении можно совершать переводы, погашать кредиты, пополнять вклады, анализировать расходы и многое другое. Клиент: Здорово! А как его можно установить? Оператор: Приложение доступно для скачивания в Play Market и App Store. После установки вам нужно будет пройти несложную регистрацию и авторизоваться, используя те же логин и пароль, что и в интернет-банке."},
    {"Негативный диалог:": "Клиент: Вашим мобильным приложением совершенно невозможно пользоваться - оно постоянно зависает и вылетает! Это ужасно! Оператор: Сожалею, что приложение доставляет вам неудобства. Мы стараемся постоянно совершенствовать его производительность и устранять замеченные недочеты. Не могли бы вы подробнее описать проблему? Это поможет разработчикам оперативно найти и исправить причину сбоев. Клиент: Да что тут описывать! Оно просто не работает нормально. То покупки не проходят, то платежи, то выдает ошибку при входе. Лучше полностью переделывайте приложение!"},
],
"Автокредиты": [
    {"Позитивный диалог:": "Клиент: Здравствуйте, меня интересуют условия автокредитования в вашем банке. Я хотел бы приобрести новую машину в кредит. Оператор: Хорошо, для покупки нового автомобиля мы можем предложить вам выгодную программу автокредитования. Максимальный срок - 5 лет, ставка от 7,9% годовых, первоначальный взнос от 20%. Также предусмотрены программы с остаточным платежом и без первого взноса. Клиент: Замечательно, спасибо. А какие документы понадобятся для оформления кредита? Оператор: Для получения автокредита необходим паспорт, водительское удостоверение и справка о доходах. При визите в отделение наш менеджер сможет помочь собрать полный пакет документов."},
    {"Негативный диалог:": "Клиент: Я крайне возмущен работой вашего банка! Мне одобрили автокредит под гораздо более высокую ставку, чем та, что озвучивалась при рекламе вашего предложения. Оператор: Сожалею, что у вас сложилось негативное впечатление. В рекламных материалах мы указываем максимально низкие ставки, действующие только для наиболее надежных клиентов с идеальной кредитной историей. Конечретные условия определяются индивидуально после рассмотрения заявки. Клиент: Это недобросовестная реклама! Вы вводите клиентов в заблуждение. Отказываюсь от вашего кредита и буду жаловаться в надзорные органы!"},
],
"Кэшбэк": [
    {"Позитивный диалог:": "Клиент: Здравствуйте, я пользуюсь вашей кредитной картой с начислением кэшбэка. Подскажите, как можно получить накопленный кэшбэк? Оператор: Приветствую! Для получения средств, накопленных по программе кэшбэка, вам нужно зайти в личный кабинет на сайте или в мобильном приложении. В разделе 'Бонусы и кэшбэк' будет указана доступная сумма для вывода на ваш счет или карту. Оформите заявку, и деньги будут зачислены в течение 3-5 рабочих дней. Клиент: Понятно, большое спасибо за разъяснение! Воспользуюсь личным кабинетом."},
    {"Негативный диалог:": "Клиент: Добрый день! Хочу высказать претензию по использованию вашей рекламной акции с кэшбэком. Мне вернули только половину обещанной суммы! Оператор: Сожалею, что возникло такое недопонимание. Давайте уточним условия акции, при которых вы рассчитывали на определенную сумму кэшбэка. Возможно, часть покупок не подпадала под действие акционного предложения согласно ограничени"}, 
]
}

ru_en_cat_names = {"Текущие (расчетные) счета": "current_acc",
"Депозиты (вклады)": "deposits",
"Кредиты": "credits",
"Кредитные карты": "credit_cards",
"Дебетовые карты": "debit_cards",
"Денежные переводы и платежи (внутренние и международные)": "payments",
"Инвестиционные продукты (брокерские услуги, ПИФы, доверительное управление)": "investments",
"Банковские гарантии и аккредитивы": "garant",
"Услуги инкассации и хранения ценностей": "colect_serv",
"Зарплатные проекты для организаций": "salary",
"Приложение банка": "bank_app",
"Автокредиты": "car_loans",
"Кэшбэк": "cashback",}
len(vocab_dialogs)

13

In [278]:
# Генерируем эмбеддинги по категориям
cat_names = []
cat_embeddings = []
pos_embeddings = []
neg_embeddings = []
for type_dialogs in vocab_dialogs:
    print(type_dialogs)
    cat_names.append(ru_en_cat_names[type_dialogs])
    pos = list(vocab_dialogs[type_dialogs][0].values())[0]
    neg = list(vocab_dialogs[type_dialogs][1].values())[0]
    # Формируем эмбеддиги
    pos_emb = bertmodel.encode(pos)
    neg_emb = bertmodel.encode(neg)
    posneg_emb = bertmodel.encode(pos+neg)
    posneg_mean_emb = np.array([pos_emb, neg_emb]).mean(axis=0)
    
    cat_embeddings.append(pos_emb)
    cat_embeddings.append(neg_emb)
    cat_embeddings.append(posneg_emb)
    cat_embeddings.append(posneg_mean_emb)
    pos_embeddings.append(pos_emb)
    neg_embeddings.append(neg_emb)

pos_embeddings = np.array(pos_embeddings)
neg_embeddings = np.array(neg_embeddings)

cat_embeddings.append(pos_embeddings.mean(axis=0))
cat_embeddings.append(neg_embeddings.mean(axis=0))
cat_embeddings = np.array(cat_embeddings)

cat_embeddings.shape

Текущие (расчетные) счета
Депозиты (вклады)
Кредиты
Кредитные карты
Дебетовые карты
Денежные переводы и платежи (внутренние и международные)
Инвестиционные продукты (брокерские услуги, ПИФы, доверительное управление)
Банковские гарантии и аккредитивы
Услуги инкассации и хранения ценностей
Зарплатные проекты для организаций
Приложение банка
Автокредиты
Кэшбэк


(54, 768)

In [282]:
# Формируем словарь названий фичей
columns_cat_names = []
for cat_name in cat_names:
    columns_cat_names.append(f'pos_{cat_name}_emb')
    columns_cat_names.append(f'neg_{cat_name}_emb')
    columns_cat_names.append(f'posneg_{cat_name}_emb')
    columns_cat_names.append(f'posneg_mean_{cat_name}_emb')        
columns_cat_names += ['positive_emb', 'negative_emb']
len(columns_cat_names)

54

In [254]:
%%time
# Производим расчет косинусного расстояния
result_cos_sim = cosine_similarity(embeds, cat_embeddings)
result_cos_sim.shape

CPU times: total: 24.2 s
Wall time: 5.52 s


(1501735, 54)

In [ ]:
# Сохраняем косинунсое расстояние в фичах для каждого диалога
all_dialogs_df[columns_cat_names] = result_cos_sim

In [284]:
all_dialogs_df = all_dialogs_df[['client_id', 'event_time', 'embedding']]
all_dialogs_df.shape

(1501735, 3)

In [ ]:
assert Falsem, "Ксоинсусное сходство для хороших и плохих проудктов"

In [296]:
%%time
# загружаем сэмплерированные данные, для которых будем рассчитывать фичи
smpl_Client_Month_df = pq.read_table(PATH_DATASET_OUTPUT + 'result_sample_Client_Month_df_12_06_2024.parquet').to_pandas()
smpl_Client_Month_df = smpl_Client_Month_df.set_index(['client_id', 'report_next_end'])
smpl_Client_Month_df.shape

CPU times: total: 1.2 s
Wall time: 1.47 s


(747847, 1)

In [297]:
# Добавляем временные колонки в сэпмплирвоанные данные, чтобы легче искать по индексу и отдельно по клиенту и по месяцу
smpl_Client_Month_df = smpl_Client_Month_df.reset_index()
smpl_Client_Month_df['col_client_id'] = smpl_Client_Month_df['client_id']
smpl_Client_Month_df['col_report_next_end'] = smpl_Client_Month_df['report_next_end']
smpl_Client_Month_df = smpl_Client_Month_df.set_index(['client_id', 'report_next_end'])
smpl_Client_Month_df.shape

(747847, 3)

### Генерируем доп фичи по косинусному сходству

In [303]:
# columns_cat_names

In [314]:
%%time
def calc_embedding_stats(data):
    data = select_mon_dlg_df

    # Группируем данные по client_id
    stats_dialogs = data.groupby('client_id').agg(
            sum_pos_current_acc_emb = ('pos_current_acc_emb', sum),
            sum_neg_current_acc_emb = ('neg_current_acc_emb', sum),
            sum_posneg_current_acc_emb = ('posneg_current_acc_emb', sum),
            sum_posneg_mean_current_acc_emb = ('posneg_mean_current_acc_emb', sum),
            sum_pos_deposits_emb = ('pos_deposits_emb', sum),
            sum_neg_deposits_emb = ('neg_deposits_emb', sum),
            sum_posneg_deposits_emb = ('posneg_deposits_emb', sum),
            sum_posneg_mean_deposits_emb = ('posneg_mean_deposits_emb', sum),
            sum_pos_credits_emb = ('pos_credits_emb', sum),
            sum_neg_credits_emb = ('neg_credits_emb', sum),
            sum_posneg_credits_emb = ('posneg_credits_emb', sum),
            sum_posneg_mean_credits_emb = ('posneg_mean_credits_emb', sum),
            sum_pos_credit_cards_emb = ('pos_credit_cards_emb', sum),
            sum_neg_credit_cards_emb = ('neg_credit_cards_emb', sum),
            sum_posneg_credit_cards_emb = ('posneg_credit_cards_emb', sum),
            sum_posneg_mean_credit_cards_emb = ('posneg_mean_credit_cards_emb', sum),
            sum_pos_debit_cards_emb = ('pos_debit_cards_emb', sum),
            sum_neg_debit_cards_emb = ('neg_debit_cards_emb', sum),
            sum_posneg_debit_cards_emb = ('posneg_debit_cards_emb', sum),
            sum_posneg_mean_debit_cards_emb = ('posneg_mean_debit_cards_emb', sum),
            sum_pos_payments_emb = ('pos_payments_emb', sum),
            sum_neg_payments_emb = ('neg_payments_emb', sum),
            sum_posneg_payments_emb = ('posneg_payments_emb', sum),
            sum_posneg_mean_payments_emb = ('posneg_mean_payments_emb', sum),
            sum_pos_investments_emb = ('pos_investments_emb', sum),
            sum_neg_investments_emb = ('neg_investments_emb', sum),
            sum_posneg_investments_emb = ('posneg_investments_emb', sum),
            sum_posneg_mean_investments_emb = ('posneg_mean_investments_emb', sum),
            sum_pos_garant_emb = ('pos_garant_emb', sum),
            sum_neg_garant_emb = ('neg_garant_emb', sum),
            sum_posneg_garant_emb = ('posneg_garant_emb', sum),
            sum_posneg_mean_garant_emb = ('posneg_mean_garant_emb', sum),
            sum_pos_colect_serv_emb = ('pos_colect_serv_emb', sum),
            sum_neg_colect_serv_emb = ('neg_colect_serv_emb', sum),
            sum_posneg_colect_serv_emb = ('posneg_colect_serv_emb', sum),
            sum_posneg_mean_colect_serv_emb = ('posneg_mean_colect_serv_emb', sum),
            sum_pos_salary_emb = ('pos_salary_emb', sum),
            sum_neg_salary_emb = ('neg_salary_emb', sum),
            sum_posneg_salary_emb = ('posneg_salary_emb', sum),
            sum_posneg_mean_salary_emb = ('posneg_mean_salary_emb', sum),
            sum_pos_bank_app_emb = ('pos_bank_app_emb', sum),
            sum_neg_bank_app_emb = ('neg_bank_app_emb', sum),
            sum_posneg_bank_app_emb = ('posneg_bank_app_emb', sum),
            sum_posneg_mean_bank_app_emb = ('posneg_mean_bank_app_emb', sum),
            sum_pos_car_loans_emb = ('pos_car_loans_emb', sum),
            sum_neg_car_loans_emb = ('neg_car_loans_emb', sum),
            sum_posneg_car_loans_emb = ('posneg_car_loans_emb', sum),
            sum_posneg_mean_car_loans_emb = ('posneg_mean_car_loans_emb', sum),
            sum_pos_cashback_emb = ('pos_cashback_emb', sum),
            sum_neg_cashback_emb = ('neg_cashback_emb', sum),
            sum_posneg_cashback_emb = ('posneg_cashback_emb', sum),
            sum_posneg_mean_cashback_emb = ('posneg_mean_cashback_emb', sum),
            sum_positive_emb = ('positive_emb', sum),
            sum_negative_emb = ('negative_emb', sum),
    
    
            mean_pos_current_acc_emb = ('pos_current_acc_emb', np.mean),
            mean_neg_current_acc_emb = ('neg_current_acc_emb', np.mean),
            mean_posneg_current_acc_emb = ('posneg_current_acc_emb', np.mean),
            mean_posneg_mean_current_acc_emb = ('posneg_mean_current_acc_emb', np.mean),
            mean_pos_deposits_emb = ('pos_deposits_emb', np.mean),
            mean_neg_deposits_emb = ('neg_deposits_emb', np.mean),
            mean_posneg_deposits_emb = ('posneg_deposits_emb', np.mean),
            mean_posneg_mean_deposits_emb = ('posneg_mean_deposits_emb', np.mean),
            mean_pos_credits_emb = ('pos_credits_emb', np.mean),
            mean_neg_credits_emb = ('neg_credits_emb', np.mean),
            mean_posneg_credits_emb = ('posneg_credits_emb', np.mean),
            mean_posneg_mean_credits_emb = ('posneg_mean_credits_emb', np.mean),
            mean_pos_credit_cards_emb = ('pos_credit_cards_emb', np.mean),
            mean_neg_credit_cards_emb = ('neg_credit_cards_emb', np.mean),
            mean_posneg_credit_cards_emb = ('posneg_credit_cards_emb', np.mean),
            mean_posneg_mean_credit_cards_emb = ('posneg_mean_credit_cards_emb', np.mean),
            mean_pos_debit_cards_emb = ('pos_debit_cards_emb', np.mean),
            mean_neg_debit_cards_emb = ('neg_debit_cards_emb', np.mean),
            mean_posneg_debit_cards_emb = ('posneg_debit_cards_emb', np.mean),
            mean_posneg_mean_debit_cards_emb = ('posneg_mean_debit_cards_emb', np.mean),
            mean_pos_payments_emb = ('pos_payments_emb', np.mean),
            mean_neg_payments_emb = ('neg_payments_emb', np.mean),
            mean_posneg_payments_emb = ('posneg_payments_emb', np.mean),
            mean_posneg_mean_payments_emb = ('posneg_mean_payments_emb', np.mean),
            mean_pos_investments_emb = ('pos_investments_emb', np.mean),
            mean_neg_investments_emb = ('neg_investments_emb', np.mean),
            mean_posneg_investments_emb = ('posneg_investments_emb', np.mean),
            mean_posneg_mean_investments_emb = ('posneg_mean_investments_emb', np.mean),
            mean_pos_garant_emb = ('pos_garant_emb', np.mean),
            mean_neg_garant_emb = ('neg_garant_emb', np.mean),
            mean_posneg_garant_emb = ('posneg_garant_emb', np.mean),
            mean_posneg_mean_garant_emb = ('posneg_mean_garant_emb', np.mean),
            mean_pos_colect_serv_emb = ('pos_colect_serv_emb', np.mean),
            mean_neg_colect_serv_emb = ('neg_colect_serv_emb', np.mean),
            mean_posneg_colect_serv_emb = ('posneg_colect_serv_emb', np.mean),
            mean_posneg_mean_colect_serv_emb = ('posneg_mean_colect_serv_emb', np.mean),
            mean_pos_salary_emb = ('pos_salary_emb', np.mean),
            mean_neg_salary_emb = ('neg_salary_emb', np.mean),
            mean_posneg_salary_emb = ('posneg_salary_emb', np.mean),
            mean_posneg_mean_salary_emb = ('posneg_mean_salary_emb', np.mean),
            mean_pos_bank_app_emb = ('pos_bank_app_emb', np.mean),
            mean_neg_bank_app_emb = ('neg_bank_app_emb', np.mean),
            mean_posneg_bank_app_emb = ('posneg_bank_app_emb', np.mean),
            mean_posneg_mean_bank_app_emb = ('posneg_mean_bank_app_emb', np.mean),
            mean_pos_car_loans_emb = ('pos_car_loans_emb', np.mean),
            mean_neg_car_loans_emb = ('neg_car_loans_emb', np.mean),
            mean_posneg_car_loans_emb = ('posneg_car_loans_emb', np.mean),
            mean_posneg_mean_car_loans_emb = ('posneg_mean_car_loans_emb', np.mean),
            mean_pos_cashback_emb = ('pos_cashback_emb', np.mean),
            mean_neg_cashback_emb = ('neg_cashback_emb', np.mean),
            mean_posneg_cashback_emb = ('posneg_cashback_emb', np.mean),
            mean_posneg_mean_cashback_emb = ('posneg_mean_cashback_emb', np.mean),
            mean_positive_emb = ('positive_emb', np.mean),
            mean_negative_emb = ('negative_emb', np.mean),

            min_pos_current_acc_emb = ('pos_current_acc_emb', min),
            min_neg_current_acc_emb = ('neg_current_acc_emb', min),
            min_posneg_current_acc_emb = ('posneg_current_acc_emb', min),
            min_posneg_mean_current_acc_emb = ('posneg_mean_current_acc_emb', min),
            min_pos_deposits_emb = ('pos_deposits_emb', min),
            min_neg_deposits_emb = ('neg_deposits_emb', min),
            min_posneg_deposits_emb = ('posneg_deposits_emb', min),
            min_posneg_mean_deposits_emb = ('posneg_mean_deposits_emb', min),
            min_pos_credits_emb = ('pos_credits_emb', min),
            min_neg_credits_emb = ('neg_credits_emb', min),
            min_posneg_credits_emb = ('posneg_credits_emb', min),
            min_posneg_mean_credits_emb = ('posneg_mean_credits_emb', min),
            min_pos_credit_cards_emb = ('pos_credit_cards_emb', min),
            min_neg_credit_cards_emb = ('neg_credit_cards_emb', min),
            min_posneg_credit_cards_emb = ('posneg_credit_cards_emb', min),
            min_posneg_mean_credit_cards_emb = ('posneg_mean_credit_cards_emb', min),
            min_pos_debit_cards_emb = ('pos_debit_cards_emb', min),
            min_neg_debit_cards_emb = ('neg_debit_cards_emb', min),
            min_posneg_debit_cards_emb = ('posneg_debit_cards_emb', min),
            min_posneg_mean_debit_cards_emb = ('posneg_mean_debit_cards_emb', min),
            min_pos_payments_emb = ('pos_payments_emb', min),
            min_neg_payments_emb = ('neg_payments_emb', min),
            min_posneg_payments_emb = ('posneg_payments_emb', min),
            min_posneg_mean_payments_emb = ('posneg_mean_payments_emb', min),
            min_pos_investments_emb = ('pos_investments_emb', min),
            min_neg_investments_emb = ('neg_investments_emb', min),
            min_posneg_investments_emb = ('posneg_investments_emb', min),
            min_posneg_mean_investments_emb = ('posneg_mean_investments_emb', min),
            min_pos_garant_emb = ('pos_garant_emb', min),
            min_neg_garant_emb = ('neg_garant_emb', min),
            min_posneg_garant_emb = ('posneg_garant_emb', min),
            min_posneg_mean_garant_emb = ('posneg_mean_garant_emb', min),
            min_pos_colect_serv_emb = ('pos_colect_serv_emb', min),
            min_neg_colect_serv_emb = ('neg_colect_serv_emb', min),
            min_posneg_colect_serv_emb = ('posneg_colect_serv_emb', min),
            min_posneg_mean_colect_serv_emb = ('posneg_mean_colect_serv_emb', min),
            min_pos_salary_emb = ('pos_salary_emb', min),
            min_neg_salary_emb = ('neg_salary_emb', min),
            min_posneg_salary_emb = ('posneg_salary_emb', min),
            min_posneg_mean_salary_emb = ('posneg_mean_salary_emb', min),
            min_pos_bank_app_emb = ('pos_bank_app_emb', min),
            min_neg_bank_app_emb = ('neg_bank_app_emb', min),
            min_posneg_bank_app_emb = ('posneg_bank_app_emb', min),
            min_posneg_mean_bank_app_emb = ('posneg_mean_bank_app_emb', min),
            min_pos_car_loans_emb = ('pos_car_loans_emb', min),
            min_neg_car_loans_emb = ('neg_car_loans_emb', min),
            min_posneg_car_loans_emb = ('posneg_car_loans_emb', min),
            min_posneg_mean_car_loans_emb = ('posneg_mean_car_loans_emb', min),
            min_pos_cashback_emb = ('pos_cashback_emb', min),
            min_neg_cashback_emb = ('neg_cashback_emb', min),
            min_posneg_cashback_emb = ('posneg_cashback_emb', min),
            min_posneg_mean_cashback_emb = ('posneg_mean_cashback_emb', min),
            min_positive_emb = ('positive_emb', min),
            min_negative_emb = ('negative_emb', min),
    
    
            max_pos_current_acc_emb = ('pos_current_acc_emb', max),
            max_neg_current_acc_emb = ('neg_current_acc_emb', max),
            max_posneg_current_acc_emb = ('posneg_current_acc_emb', max),
            max_posneg_mean_current_acc_emb = ('posneg_mean_current_acc_emb', max),
            max_pos_deposits_emb = ('pos_deposits_emb', max),
            max_neg_deposits_emb = ('neg_deposits_emb', max),
            max_posneg_deposits_emb = ('posneg_deposits_emb', max),
            max_posneg_mean_deposits_emb = ('posneg_mean_deposits_emb', max),
            max_pos_credits_emb = ('pos_credits_emb', max),
            max_neg_credits_emb = ('neg_credits_emb', max),
            max_posneg_credits_emb = ('posneg_credits_emb', max),
            max_posneg_mean_credits_emb = ('posneg_mean_credits_emb', max),
            max_pos_credit_cards_emb = ('pos_credit_cards_emb', max),
            max_neg_credit_cards_emb = ('neg_credit_cards_emb', max),
            max_posneg_credit_cards_emb = ('posneg_credit_cards_emb', max),
            max_posneg_mean_credit_cards_emb = ('posneg_mean_credit_cards_emb', max),
            max_pos_debit_cards_emb = ('pos_debit_cards_emb', max),
            max_neg_debit_cards_emb = ('neg_debit_cards_emb', max),
            max_posneg_debit_cards_emb = ('posneg_debit_cards_emb', max),
            max_posneg_mean_debit_cards_emb = ('posneg_mean_debit_cards_emb', max),
            max_pos_payments_emb = ('pos_payments_emb', max),
            max_neg_payments_emb = ('neg_payments_emb', max),
            max_posneg_payments_emb = ('posneg_payments_emb', max),
            max_posneg_mean_payments_emb = ('posneg_mean_payments_emb', max),
            max_pos_investments_emb = ('pos_investments_emb', max),
            max_neg_investments_emb = ('neg_investments_emb', max),
            max_posneg_investments_emb = ('posneg_investments_emb', max),
            max_posneg_mean_investments_emb = ('posneg_mean_investments_emb', max),
            max_pos_garant_emb = ('pos_garant_emb', max),
            max_neg_garant_emb = ('neg_garant_emb', max),
            max_posneg_garant_emb = ('posneg_garant_emb', max),
            max_posneg_mean_garant_emb = ('posneg_mean_garant_emb', max),
            max_pos_colect_serv_emb = ('pos_colect_serv_emb', max),
            max_neg_colect_serv_emb = ('neg_colect_serv_emb', max),
            max_posneg_colect_serv_emb = ('posneg_colect_serv_emb', max),
            max_posneg_mean_colect_serv_emb = ('posneg_mean_colect_serv_emb', max),
            max_pos_salary_emb = ('pos_salary_emb', max),
            max_neg_salary_emb = ('neg_salary_emb', max),
            max_posneg_salary_emb = ('posneg_salary_emb', max),
            max_posneg_mean_salary_emb = ('posneg_mean_salary_emb', max),
            max_pos_bank_app_emb = ('pos_bank_app_emb', max),
            max_neg_bank_app_emb = ('neg_bank_app_emb', max),
            max_posneg_bank_app_emb = ('posneg_bank_app_emb', max),
            max_posneg_mean_bank_app_emb = ('posneg_mean_bank_app_emb', max),
            max_pos_car_loans_emb = ('pos_car_loans_emb', max),
            max_neg_car_loans_emb = ('neg_car_loans_emb', max),
            max_posneg_car_loans_emb = ('posneg_car_loans_emb', max),
            max_posneg_mean_car_loans_emb = ('posneg_mean_car_loans_emb', max),
            max_pos_cashback_emb = ('pos_cashback_emb', max),
            max_neg_cashback_emb = ('neg_cashback_emb', max),
            max_posneg_cashback_emb = ('posneg_cashback_emb', max),
            max_posneg_mean_cashback_emb = ('posneg_mean_cashback_emb', max),
            max_positive_emb = ('positive_emb', max),
            max_negative_emb = ('negative_emb', max),    
    
            std_pos_current_acc_emb = ('pos_current_acc_emb', np.std),
            std_neg_current_acc_emb = ('neg_current_acc_emb', np.std),
            std_posneg_current_acc_emb = ('posneg_current_acc_emb', np.std),
            std_posneg_mean_current_acc_emb = ('posneg_mean_current_acc_emb', np.std),
            std_pos_deposits_emb = ('pos_deposits_emb', np.std),
            std_neg_deposits_emb = ('neg_deposits_emb', np.std),
            std_posneg_deposits_emb = ('posneg_deposits_emb', np.std),
            std_posneg_mean_deposits_emb = ('posneg_mean_deposits_emb', np.std),
            std_pos_credits_emb = ('pos_credits_emb', np.std),
            std_neg_credits_emb = ('neg_credits_emb', np.std),
            std_posneg_credits_emb = ('posneg_credits_emb', np.std),
            std_posneg_mean_credits_emb = ('posneg_mean_credits_emb', np.std),
            std_pos_credit_cards_emb = ('pos_credit_cards_emb', np.std),
            std_neg_credit_cards_emb = ('neg_credit_cards_emb', np.std),
            std_posneg_credit_cards_emb = ('posneg_credit_cards_emb', np.std),
            std_posneg_mean_credit_cards_emb = ('posneg_mean_credit_cards_emb', np.std),
            std_pos_debit_cards_emb = ('pos_debit_cards_emb', np.std),
            std_neg_debit_cards_emb = ('neg_debit_cards_emb', np.std),
            std_posneg_debit_cards_emb = ('posneg_debit_cards_emb', np.std),
            std_posneg_mean_debit_cards_emb = ('posneg_mean_debit_cards_emb', np.std),
            std_pos_payments_emb = ('pos_payments_emb', np.std),
            std_neg_payments_emb = ('neg_payments_emb', np.std),
            std_posneg_payments_emb = ('posneg_payments_emb', np.std),
            std_posneg_mean_payments_emb = ('posneg_mean_payments_emb', np.std),
            std_pos_investments_emb = ('pos_investments_emb', np.std),
            std_neg_investments_emb = ('neg_investments_emb', np.std),
            std_posneg_investments_emb = ('posneg_investments_emb', np.std),
            std_posneg_mean_investments_emb = ('posneg_mean_investments_emb', np.std),
            std_pos_garant_emb = ('pos_garant_emb', np.std),
            std_neg_garant_emb = ('neg_garant_emb', np.std),
            std_posneg_garant_emb = ('posneg_garant_emb', np.std),
            std_posneg_mean_garant_emb = ('posneg_mean_garant_emb', np.std),
            std_pos_colect_serv_emb = ('pos_colect_serv_emb', np.std),
            std_neg_colect_serv_emb = ('neg_colect_serv_emb', np.std),
            std_posneg_colect_serv_emb = ('posneg_colect_serv_emb', np.std),
            std_posneg_mean_colect_serv_emb = ('posneg_mean_colect_serv_emb', np.std),
            std_pos_salary_emb = ('pos_salary_emb', np.std),
            std_neg_salary_emb = ('neg_salary_emb', np.std),
            std_posneg_salary_emb = ('posneg_salary_emb', np.std),
            std_posneg_mean_salary_emb = ('posneg_mean_salary_emb', np.std),
            std_pos_bank_app_emb = ('pos_bank_app_emb', np.std),
            std_neg_bank_app_emb = ('neg_bank_app_emb', np.std),
            std_posneg_bank_app_emb = ('posneg_bank_app_emb', np.std),
            std_posneg_mean_bank_app_emb = ('posneg_mean_bank_app_emb', np.std),
            std_pos_car_loans_emb = ('pos_car_loans_emb', np.std),
            std_neg_car_loans_emb = ('neg_car_loans_emb', np.std),
            std_posneg_car_loans_emb = ('posneg_car_loans_emb', np.std),
            std_posneg_mean_car_loans_emb = ('posneg_mean_car_loans_emb', np.std),
            std_pos_cashback_emb = ('pos_cashback_emb', np.std),
            std_neg_cashback_emb = ('neg_cashback_emb', np.std),
            std_posneg_cashback_emb = ('posneg_cashback_emb', np.std),
            std_posneg_mean_cashback_emb = ('posneg_mean_cashback_emb', np.std),
            std_positive_emb = ('positive_emb', np.std),
            std_negative_emb = ('negative_emb', np.std),    
            )
    return stats_dialogs.reset_index()

CPU times: total: 0 ns
Wall time: 0 ns


In [315]:
%%time
# Формируем фичи по таргету
start_date = datetime(2022, 1, 1, 0, 0, 0)
# end_date = datetime(2023, 1, 1, 0, 0, 0)
end_date = all_dialogs_df['event_time'].max()

# Итоговый датасет 
union_cos_dialogs_by_clients_df = pd.DataFrame()

# Бежим по месяцам и расчитываем статистики для клиента берем предыдущие месяцы
for i in trange(((end_date - start_date).days//30 + 1)):
    end_date = start_date + relativedelta(months=1) - relativedelta(days=1)
    print(f'start: {start_date}, end: {end_date}')
    # Начальная дата за прошедшие полгода
    begin_date = end_date - relativedelta(months=6) - relativedelta(days=1)
    # Определяем только тех клиентов которые есть в сэмлере для указанной отчетной даты
    report_next_end = start_date + relativedelta(months=2) - relativedelta(days=1)
    good_slct_clients = smpl_Client_Month_df[smpl_Client_Month_df['col_report_next_end'] == report_next_end]['col_client_id'].unique()

    # Берем данные за последний месяц и фильтруем по нужным клиентам 
    select_mon_dlg_df = all_dialogs_df[all_dialogs_df['event_time'].between(start_date, end_date)]
    select_mon_dlg_df = select_mon_dlg_df[select_mon_dlg_df['client_id'].isin(good_slct_clients)]

    # Берем данные за последние полгода начиная от даты begin_date и фильтруем по нужным клиентам 
    select_ftime_dlg_df = all_dialogs_df[all_dialogs_df['event_time'].between(begin_date, end_date)]
    select_ftime_dlg_df = select_ftime_dlg_df[select_ftime_dlg_df['client_id'].isin(good_slct_clients)]
    
    print(select_mon_dlg_df.shape, select_ftime_dlg_df.shape)
    
    
#     select_mon_current_df = all_dialogs_df[all_dialogs_df['event_time'].between(start_date, end_date)]
#     select_mon_full_df = all_dialogs_df[all_dialogs_df['event_time'].between(begin_date, end_date)]
#     print(select_mon_current_df.shape, select_mon_full_df.shape)
    
    # client_agg_df = uniq_clients_df.copy()
#     client_agg_df = select_mon_full_df[['client_id']].drop_duplicates().copy()
#     report_next_end = start_date + relativedelta(months=2) - relativedelta(days=1)
#     client_agg_df['report_next_end'] = report_next_end
#     client_agg_df = client_agg_df.set_index('client_id')    

    pop_mon_stat_dialogs_df = calc_embedding_stats(select_mon_dlg_df).fillna(0)
    pop_mon_stat_dialogs_df['report_next_end'] = report_next_end
    pop_mon_stat_dialogs_df = pop_mon_stat_dialogs_df.set_index(['client_id', 'report_next_end']).add_prefix('dlg_')
    
    pop_ftime_stat_dialogs_df = calc_embedding_stats(select_ftime_dlg_df).fillna(0)
    pop_ftime_stat_dialogs_df['report_next_end'] = report_next_end
    pop_ftime_stat_dialogs_df = pop_ftime_stat_dialogs_df.set_index(['client_id', 'report_next_end']).add_prefix('ftime_dlg_')
    
    # Объединяем фичи по месяцам и за полгода
    pop_ftime_stat_dialogs_df = pop_ftime_stat_dialogs_df.merge(pop_mon_stat_dialogs_df, left_index=True, right_index=True, how='left')
    
    # Сводим в единый датафрейм
    union_cos_dialogs_by_clients_df = pd.concat([union_cos_dialogs_by_clients_df, pop_ftime_stat_dialogs_df])
    
    start_date = start_date + relativedelta(months=1)
union_cos_dialogs_by_clients_df.shape

  0%|          | 0/13 [00:00<?, ?it/s]

start: 2022-01-01 00:00:00, end: 2022-01-31 00:00:00
(5377, 57) (5378, 57)


  8%|▊         | 1/13 [00:00<00:03,  3.33it/s]

start: 2022-02-01 00:00:00, end: 2022-02-28 00:00:00
(4785, 57) (9415, 57)


 15%|█▌        | 2/13 [00:00<00:03,  3.44it/s]

...

 92%|█████████▏| 12/13 [00:06<00:00,  1.43it/s]

start: 2023-01-01 00:00:00, end: 2023-01-31 00:00:00
(0, 57) (0, 57)


100%|██████████| 13/13 [00:06<00:00,  1.94it/s]

CPU times: total: 6.61 s
Wall time: 6.7 s


(71877, 540)

In [319]:
%%time
# Сохраняем статистику по косинусной близости
union_cos_dialogs_by_clients_df.to_parquet(PATH_DATASET_OUTPUT + 'cos_dialogs_by_clients_df_13_06_2024.parquet')

CPU times: total: 2.19 s
Wall time: 2.08 s


### Разбивка эмбеддинга на подгруппы и рассчет статистик отдельно для них 

In [393]:
length_embedding = 768
part_size = 64 # должно быть одно из делителей 768: 1, 2, 3, 4, 6, 8, 12, 16, 24, 32, 48, 64, 96, 128, 192, 256, 384, 768
count_parts_emb = math.ceil(768/part_size)
columns_part = []
for i in range(count_parts_emb):
    columns_part.append(f'{i+1}_part_emb')
len(columns_part)

12

In [363]:
all_dialogs_df = all_dialogs_df[['client_id', 'event_time', 'embedding']]
all_dialogs_df.shape

(1501735, 3)

In [401]:
def calc_part_emb_stats(data):
    # Группируем данные по client_id
    stats_part_emds = data.groupby('client_id').agg(
        min__min_1_part_emb = ('min_1_part_emb', min),
        min__max_1_part_emb = ('max_1_part_emb', min),
        min__sum_1_part_emb = ('sum_1_part_emb', min),
        min__std_1_part_emb = ('std_1_part_emb', min),
        min__mean_1_part_emb = ('mean_1_part_emb', min),
        min__min_2_part_emb = ('min_2_part_emb', min),
        min__max_2_part_emb = ('max_2_part_emb', min),
        min__sum_2_part_emb = ('sum_2_part_emb', min),
        min__std_2_part_emb = ('std_2_part_emb', min),
        min__mean_2_part_emb = ('mean_2_part_emb', min),
        min__min_3_part_emb = ('min_3_part_emb', min),
        min__max_3_part_emb = ('max_3_part_emb', min),
        min__sum_3_part_emb = ('sum_3_part_emb', min),
        min__std_3_part_emb = ('std_3_part_emb', min),
        min__mean_3_part_emb = ('mean_3_part_emb', min),
        min__min_4_part_emb = ('min_4_part_emb', min),
        min__max_4_part_emb = ('max_4_part_emb', min),
        min__sum_4_part_emb = ('sum_4_part_emb', min),
        min__std_4_part_emb = ('std_4_part_emb', min),
        min__mean_4_part_emb = ('mean_4_part_emb', min),
        min__min_5_part_emb = ('min_5_part_emb', min),
        min__max_5_part_emb = ('max_5_part_emb', min),
        min__sum_5_part_emb = ('sum_5_part_emb', min),
        min__std_5_part_emb = ('std_5_part_emb', min),
        min__mean_5_part_emb = ('mean_5_part_emb', min),
        min__min_6_part_emb = ('min_6_part_emb', min),
        min__max_6_part_emb = ('max_6_part_emb', min),
        min__sum_6_part_emb = ('sum_6_part_emb', min),
        min__std_6_part_emb = ('std_6_part_emb', min),
        min__mean_6_part_emb = ('mean_6_part_emb', min),
        min__min_7_part_emb = ('min_7_part_emb', min),
        min__max_7_part_emb = ('max_7_part_emb', min),
        min__sum_7_part_emb = ('sum_7_part_emb', min),
        min__std_7_part_emb = ('std_7_part_emb', min),
        min__mean_7_part_emb = ('mean_7_part_emb', min),
        min__min_8_part_emb = ('min_8_part_emb', min),
        min__max_8_part_emb = ('max_8_part_emb', min),
        min__sum_8_part_emb = ('sum_8_part_emb', min),
        min__std_8_part_emb = ('std_8_part_emb', min),
        min__mean_8_part_emb = ('mean_8_part_emb', min),
        min__min_9_part_emb = ('min_9_part_emb', min),
        min__max_9_part_emb = ('max_9_part_emb', min),
        min__sum_9_part_emb = ('sum_9_part_emb', min),
        min__std_9_part_emb = ('std_9_part_emb', min),
        min__mean_9_part_emb = ('mean_9_part_emb', min),
        min__min_10_part_emb = ('min_10_part_emb', min),
        min__max_10_part_emb = ('max_10_part_emb', min),
        min__sum_10_part_emb = ('sum_10_part_emb', min),
        min__std_10_part_emb = ('std_10_part_emb', min),
        min__mean_10_part_emb = ('mean_10_part_emb', min),
        min__min_11_part_emb = ('min_11_part_emb', min),
        min__max_11_part_emb = ('max_11_part_emb', min),
        min__sum_11_part_emb = ('sum_11_part_emb', min),
        min__std_11_part_emb = ('std_11_part_emb', min),
        min__mean_11_part_emb = ('mean_11_part_emb', min),
        min__min_12_part_emb = ('min_12_part_emb', min),
        min__max_12_part_emb = ('max_12_part_emb', min),
        min__sum_12_part_emb = ('sum_12_part_emb', min),
        min__std_12_part_emb = ('std_12_part_emb', min),
        min__mean_12_part_emb = ('mean_12_part_emb', min),

        max__min_1_part_emb = ('min_1_part_emb', max),
        max__max_1_part_emb = ('max_1_part_emb', max),
        max__sum_1_part_emb = ('sum_1_part_emb', max),
        max__std_1_part_emb = ('std_1_part_emb', max),
        max__mean_1_part_emb = ('mean_1_part_emb', max),
        max__min_2_part_emb = ('min_2_part_emb', max),
        max__max_2_part_emb = ('max_2_part_emb', max),
        max__sum_2_part_emb = ('sum_2_part_emb', max),
        max__std_2_part_emb = ('std_2_part_emb', max),
        max__mean_2_part_emb = ('mean_2_part_emb', max),
        max__min_3_part_emb = ('min_3_part_emb', max),
        max__max_3_part_emb = ('max_3_part_emb', max),
        max__sum_3_part_emb = ('sum_3_part_emb', max),
        max__std_3_part_emb = ('std_3_part_emb', max),
        max__mean_3_part_emb = ('mean_3_part_emb', max),
        max__min_4_part_emb = ('min_4_part_emb', max),
        max__max_4_part_emb = ('max_4_part_emb', max),
        max__sum_4_part_emb = ('sum_4_part_emb', max),
        max__std_4_part_emb = ('std_4_part_emb', max),
        max__mean_4_part_emb = ('mean_4_part_emb', max),
        max__min_5_part_emb = ('min_5_part_emb', max),
        max__max_5_part_emb = ('max_5_part_emb', max),
        max__sum_5_part_emb = ('sum_5_part_emb', max),
        max__std_5_part_emb = ('std_5_part_emb', max),
        max__mean_5_part_emb = ('mean_5_part_emb', max),
        max__min_6_part_emb = ('min_6_part_emb', max),
        max__max_6_part_emb = ('max_6_part_emb', max),
        max__sum_6_part_emb = ('sum_6_part_emb', max),
        max__std_6_part_emb = ('std_6_part_emb', max),
        max__mean_6_part_emb = ('mean_6_part_emb', max),
        max__min_7_part_emb = ('min_7_part_emb', max),
        max__max_7_part_emb = ('max_7_part_emb', max),
        max__sum_7_part_emb = ('sum_7_part_emb', max),
        max__std_7_part_emb = ('std_7_part_emb', max),
        max__mean_7_part_emb = ('mean_7_part_emb', max),
        max__min_8_part_emb = ('min_8_part_emb', max),
        max__max_8_part_emb = ('max_8_part_emb', max),
        max__sum_8_part_emb = ('sum_8_part_emb', max),
        max__std_8_part_emb = ('std_8_part_emb', max),
        max__mean_8_part_emb = ('mean_8_part_emb', max),
        max__min_9_part_emb = ('min_9_part_emb', max),
        max__max_9_part_emb = ('max_9_part_emb', max),
        max__sum_9_part_emb = ('sum_9_part_emb', max),
        max__std_9_part_emb = ('std_9_part_emb', max),
        max__mean_9_part_emb = ('mean_9_part_emb', max),
        max__min_10_part_emb = ('min_10_part_emb', max),
        max__max_10_part_emb = ('max_10_part_emb', max),
        max__sum_10_part_emb = ('sum_10_part_emb', max),
        max__std_10_part_emb = ('std_10_part_emb', max),
        max__mean_10_part_emb = ('mean_10_part_emb', max),
        max__min_11_part_emb = ('min_11_part_emb', max),
        max__max_11_part_emb = ('max_11_part_emb', max),
        max__sum_11_part_emb = ('sum_11_part_emb', max),
        max__std_11_part_emb = ('std_11_part_emb', max),
        max__mean_11_part_emb = ('mean_11_part_emb', max),
        max__min_12_part_emb = ('min_12_part_emb', max),
        max__max_12_part_emb = ('max_12_part_emb', max),
        max__sum_12_part_emb = ('sum_12_part_emb', max),
        max__std_12_part_emb = ('std_12_part_emb', max),
        max__mean_12_part_emb = ('mean_12_part_emb', max),

        mean__min_1_part_emb = ('min_1_part_emb', np.mean),
        mean__max_1_part_emb = ('max_1_part_emb', np.mean),
        mean__sum_1_part_emb = ('sum_1_part_emb', np.mean),
        mean__std_1_part_emb = ('std_1_part_emb', np.mean),
        mean__mean_1_part_emb = ('mean_1_part_emb', np.mean),
        mean__min_2_part_emb = ('min_2_part_emb', np.mean),
        mean__max_2_part_emb = ('max_2_part_emb', np.mean),
        mean__sum_2_part_emb = ('sum_2_part_emb', np.mean),
        mean__std_2_part_emb = ('std_2_part_emb', np.mean),
        mean__mean_2_part_emb = ('mean_2_part_emb', np.mean),
        mean__min_3_part_emb = ('min_3_part_emb', np.mean),
        mean__max_3_part_emb = ('max_3_part_emb', np.mean),
        mean__sum_3_part_emb = ('sum_3_part_emb', np.mean),
        mean__std_3_part_emb = ('std_3_part_emb', np.mean),
        mean__mean_3_part_emb = ('mean_3_part_emb', np.mean),
        mean__min_4_part_emb = ('min_4_part_emb', np.mean),
        mean__max_4_part_emb = ('max_4_part_emb', np.mean),
        mean__sum_4_part_emb = ('sum_4_part_emb', np.mean),
        mean__std_4_part_emb = ('std_4_part_emb', np.mean),
        mean__mean_4_part_emb = ('mean_4_part_emb', np.mean),
        mean__min_5_part_emb = ('min_5_part_emb', np.mean),
        mean__max_5_part_emb = ('max_5_part_emb', np.mean),
        mean__sum_5_part_emb = ('sum_5_part_emb', np.mean),
        mean__std_5_part_emb = ('std_5_part_emb', np.mean),
        mean__mean_5_part_emb = ('mean_5_part_emb', np.mean),
        mean__min_6_part_emb = ('min_6_part_emb', np.mean),
        mean__max_6_part_emb = ('max_6_part_emb', np.mean),
        mean__sum_6_part_emb = ('sum_6_part_emb', np.mean),
        mean__std_6_part_emb = ('std_6_part_emb', np.mean),
        mean__mean_6_part_emb = ('mean_6_part_emb', np.mean),
        mean__min_7_part_emb = ('min_7_part_emb', np.mean),
        mean__max_7_part_emb = ('max_7_part_emb', np.mean),
        mean__sum_7_part_emb = ('sum_7_part_emb', np.mean),
        mean__std_7_part_emb = ('std_7_part_emb', np.mean),
        mean__mean_7_part_emb = ('mean_7_part_emb', np.mean),
        mean__min_8_part_emb = ('min_8_part_emb', np.mean),
        mean__max_8_part_emb = ('max_8_part_emb', np.mean),
        mean__sum_8_part_emb = ('sum_8_part_emb', np.mean),
        mean__std_8_part_emb = ('std_8_part_emb', np.mean),
        mean__mean_8_part_emb = ('mean_8_part_emb', np.mean),
        mean__min_9_part_emb = ('min_9_part_emb', np.mean),
        mean__max_9_part_emb = ('max_9_part_emb', np.mean),
        mean__sum_9_part_emb = ('sum_9_part_emb', np.mean),
        mean__std_9_part_emb = ('std_9_part_emb', np.mean),
        mean__mean_9_part_emb = ('mean_9_part_emb', np.mean),
        mean__min_10_part_emb = ('min_10_part_emb', np.mean),
        mean__max_10_part_emb = ('max_10_part_emb', np.mean),
        mean__sum_10_part_emb = ('sum_10_part_emb', np.mean),
        mean__std_10_part_emb = ('std_10_part_emb', np.mean),
        mean__mean_10_part_emb = ('mean_10_part_emb', np.mean),
        mean__min_11_part_emb = ('min_11_part_emb', np.mean),
        mean__max_11_part_emb = ('max_11_part_emb', np.mean),
        mean__sum_11_part_emb = ('sum_11_part_emb', np.mean),
        mean__std_11_part_emb = ('std_11_part_emb', np.mean),
        mean__mean_11_part_emb = ('mean_11_part_emb', np.mean),
        mean__min_12_part_emb = ('min_12_part_emb', np.mean),
        mean__max_12_part_emb = ('max_12_part_emb', np.mean),
        mean__sum_12_part_emb = ('sum_12_part_emb', np.mean),
        mean__std_12_part_emb = ('std_12_part_emb', np.mean),
        mean__mean_12_part_emb = ('mean_12_part_emb', np.mean),

        sum__min_1_part_emb = ('min_1_part_emb', sum),
        sum__max_1_part_emb = ('max_1_part_emb', sum),
        sum__sum_1_part_emb = ('sum_1_part_emb', sum),
        sum__std_1_part_emb = ('std_1_part_emb', sum),
        sum__mean_1_part_emb = ('mean_1_part_emb', sum),
        sum__min_2_part_emb = ('min_2_part_emb', sum),
        sum__max_2_part_emb = ('max_2_part_emb', sum),
        sum__sum_2_part_emb = ('sum_2_part_emb', sum),
        sum__std_2_part_emb = ('std_2_part_emb', sum),
        sum__mean_2_part_emb = ('mean_2_part_emb', sum),
        sum__min_3_part_emb = ('min_3_part_emb', sum),
        sum__max_3_part_emb = ('max_3_part_emb', sum),
        sum__sum_3_part_emb = ('sum_3_part_emb', sum),
        sum__std_3_part_emb = ('std_3_part_emb', sum),
        sum__mean_3_part_emb = ('mean_3_part_emb', sum),
        sum__min_4_part_emb = ('min_4_part_emb', sum),
        sum__max_4_part_emb = ('max_4_part_emb', sum),
        sum__sum_4_part_emb = ('sum_4_part_emb', sum),
        sum__std_4_part_emb = ('std_4_part_emb', sum),
        sum__mean_4_part_emb = ('mean_4_part_emb', sum),
        sum__min_5_part_emb = ('min_5_part_emb', sum),
        sum__max_5_part_emb = ('max_5_part_emb', sum),
        sum__sum_5_part_emb = ('sum_5_part_emb', sum),
        sum__std_5_part_emb = ('std_5_part_emb', sum),
        sum__mean_5_part_emb = ('mean_5_part_emb', sum),
        sum__min_6_part_emb = ('min_6_part_emb', sum),
        sum__max_6_part_emb = ('max_6_part_emb', sum),
        sum__sum_6_part_emb = ('sum_6_part_emb', sum),
        sum__std_6_part_emb = ('std_6_part_emb', sum),
        sum__mean_6_part_emb = ('mean_6_part_emb', sum),
        sum__min_7_part_emb = ('min_7_part_emb', sum),
        sum__max_7_part_emb = ('max_7_part_emb', sum),
        sum__sum_7_part_emb = ('sum_7_part_emb', sum),
        sum__std_7_part_emb = ('std_7_part_emb', sum),
        sum__mean_7_part_emb = ('mean_7_part_emb', sum),
        sum__min_8_part_emb = ('min_8_part_emb', sum),
        sum__max_8_part_emb = ('max_8_part_emb', sum),
        sum__sum_8_part_emb = ('sum_8_part_emb', sum),
        sum__std_8_part_emb = ('std_8_part_emb', sum),
        sum__mean_8_part_emb = ('mean_8_part_emb', sum),
        sum__min_9_part_emb = ('min_9_part_emb', sum),
        sum__max_9_part_emb = ('max_9_part_emb', sum),
        sum__sum_9_part_emb = ('sum_9_part_emb', sum),
        sum__std_9_part_emb = ('std_9_part_emb', sum),
        sum__mean_9_part_emb = ('mean_9_part_emb', sum),
        sum__min_10_part_emb = ('min_10_part_emb', sum),
        sum__max_10_part_emb = ('max_10_part_emb', sum),
        sum__sum_10_part_emb = ('sum_10_part_emb', sum),
        sum__std_10_part_emb = ('std_10_part_emb', sum),
        sum__mean_10_part_emb = ('mean_10_part_emb', sum),
        sum__min_11_part_emb = ('min_11_part_emb', sum),
        sum__max_11_part_emb = ('max_11_part_emb', sum),
        sum__sum_11_part_emb = ('sum_11_part_emb', sum),
        sum__std_11_part_emb = ('std_11_part_emb', sum),
        sum__mean_11_part_emb = ('mean_11_part_emb', sum),
        sum__min_12_part_emb = ('min_12_part_emb', sum),
        sum__max_12_part_emb = ('max_12_part_emb', sum),
        sum__sum_12_part_emb = ('sum_12_part_emb', sum),
        sum__std_12_part_emb = ('std_12_part_emb', sum),
        sum__mean_12_part_emb = ('mean_12_part_emb', sum),        
    )

    return stats_part_emds.reset_index()


In [414]:
%%time
# Формируем фичи по таргету
start_date = datetime(2022, 1, 1, 0, 0, 0)
end_date = all_dialogs_df['event_time'].max()

# Итоговый датасет 
union_split_emb_dialogs_by_clients_df = pd.DataFrame()

def split_emb(x):
    split_embeddings = [x[i*part_size:(i+1)*part_size] for i in range(count_parts_emb)]
    return split_embeddings

def calc_stat_by_one_emb(data):
    for col in columns_part:
        data[f'min_{col}'] = data[col].apply(lambda x: np.min(x, axis=0))
        data[f'max_{col}'] = data[col].apply(lambda x: np.max(x, axis=0))
        data[f'sum_{col}'] = data[col].apply(lambda x: np.sum(x, axis=0))
        data[f'std_{col}'] = data[col].apply(lambda x: np.std(x, axis=0))
        data[f'mean_{col}'] = data[col].apply(lambda x: np.mean(x, axis=0))    
    return data

# Бежим по месяцам и расчитываем статистики для клиента берем предыдущие месяцы
for i in trange(((end_date - start_date).days//30 + 1)):
    end_date = start_date + relativedelta(months=1) - relativedelta(days=1)
    print(f'start: {start_date}, end: {end_date}')
    # Начальная дата за прошедшие полгода
    begin_date = end_date - relativedelta(months=6) - relativedelta(days=1)
    # Определяем только тех клиентов которые есть в сэмлере для указанной отчетной даты
    report_next_end = start_date + relativedelta(months=2) - relativedelta(days=1)
    good_slct_clients = smpl_Client_Month_df[smpl_Client_Month_df['col_report_next_end'] == report_next_end]['col_client_id'].unique()

    # Берем данные за последний месяц и фильтруем по нужным клиентам 
    select_mon_dlg_df = all_dialogs_df[all_dialogs_df['event_time'].between(start_date, end_date)]
    select_mon_dlg_df = select_mon_dlg_df[select_mon_dlg_df['client_id'].isin(good_slct_clients)]

    # Берем данные за последние полгода начиная от даты begin_date и фильтруем по нужным клиентам 
    select_ftime_dlg_df = all_dialogs_df[all_dialogs_df['event_time'].between(begin_date, end_date)]
    select_ftime_dlg_df = select_ftime_dlg_df[select_ftime_dlg_df['client_id'].isin(good_slct_clients)]

    if len(select_ftime_dlg_df) == 0:
        start_date = start_date + relativedelta(months=1)
        continue

    print(select_mon_dlg_df.shape, select_ftime_dlg_df.shape)
    # stat_part_emb_dialogs_df = calc_part_emb_stats(select_mon_dlg_df).fillna(0)

    select_mon_dlg_df[columns_part] = select_mon_dlg_df['embedding'].apply(lambda x: pd.Series(split_emb(x)))    
    select_mon_dlg_df = calc_stat_by_one_emb(select_mon_dlg_df)
    
    mon_part_emb_stats_df = calc_part_emb_stats(select_mon_dlg_df).fillna(0)
    mon_part_emb_stats_df['report_next_end'] = report_next_end
    mon_part_emb_stats_df = mon_part_emb_stats_df.set_index(['client_id', 'report_next_end']).add_prefix('dlg_')

    
    select_ftime_dlg_df[columns_part] = select_ftime_dlg_df['embedding'].apply(lambda x: pd.Series(split_emb(x)))    
    select_ftime_dlg_df = calc_stat_by_one_emb(select_ftime_dlg_df)
    
    ftime_part_emb_stats_df = calc_part_emb_stats(select_ftime_dlg_df).fillna(0)
    ftime_part_emb_stats_df['report_next_end'] = report_next_end
    ftime_part_emb_stats_df = ftime_part_emb_stats_df.set_index(['client_id', 'report_next_end']).add_prefix('ftime_dlg_')

    # Объединяем фичи по месяцам и за полгода
    ftime_part_emb_stats_df = ftime_part_emb_stats_df.merge(mon_part_emb_stats_df, left_index=True, right_index=True, how='left')
    
    # Сводим в единый датафрейм
    union_split_emb_dialogs_by_clients_df = pd.concat([union_split_emb_dialogs_by_clients_df, ftime_part_emb_stats_df])
    
    start_date = start_date + relativedelta(months=1)
union_split_emb_dialogs_by_clients_df.shape

  0%|          | 0/13 [00:00<?, ?it/s]

start: 2022-01-01 00:00:00, end: 2022-01-31 00:00:00
(5377, 3) (5378, 3)


  8%|▊         | 1/13 [00:03<00:45,  3.79s/it]

start: 2022-02-01 00:00:00, end: 2022-02-28 00:00:00
(4785, 3) (9415, 3)


 15%|█▌        | 2/13 [00:08<00:50,  4.62s/it]

...

100%|██████████| 13/13 [03:24<00:00, 19.04s/it]

start: 2023-01-01 00:00:00, end: 2023-01-31 00:00:00


100%|██████████| 13/13 [03:24<00:00, 15.72s/it]

CPU times: total: 3min 24s
Wall time: 3min 24s


(157883, 480)

In [423]:
np.array(all_dialogs_df['embedding'].values).shape

(1501735,)

In [ ]:
np.vstack(tmp_dialogs_df.apply(np.array).values)

In [410]:
# ftime_part_emb_stats_df.shape
# union_split_emb_dialogs_by_clients_df.shape

(157883, 480)

In [415]:
%%time
# Сохраняем статистику по косинусной близости
union_split_emb_dialogs_by_clients_df.to_parquet(PATH_DATASET_OUTPUT + 'part_embed_dialogs_by_clients_df_13_06_2024.parquet')

CPU times: total: 4.69 s
Wall time: 4.5 s


In [406]:
# list(select_mon_dlg_df.columns)
# mon_part_emb_stats_df
union_split_emb_dialogs_by_clients_df.shape

(3621, 480)

In [366]:
tmp_df = select_ftime_dlg_df.copy()
def split_emb(x):
    split_embeddings = [x[i*part_size:(i+1)*part_size] for i in range(count_parts_emb)]
    return split_embeddings

tmp_df[columns_part] = tmp_df['embedding'].apply(lambda x: pd.Series(split_emb(x)))

In [381]:
tmp_df

,client_id,event_time,embedding,1_part_emb,2_part_emb,3_part_emb,4_part_emb,5_part_emb,6_part_emb,7_part_emb,8_part_emb,9_part_emb,10_part_emb,11_part_emb,12_part_emb,13_part_emb,14_part_emb,15_part_emb,16_part_emb,17_part_emb,18_part_emb,19_part_emb,20_part_emb,21_part_emb,22_part_emb,23_part_emb,24_part_emb,25_part_emb,26_part_emb,27_part_emb,28_part_emb,29_part_emb,30_part_emb,31_part_emb,32_part_emb
28,a138dca7063cc5e1b85b91d8ab125f723bf9fe980cb9fb...,2022-01-06 03:41:47.104645,"[0.18779568, -0.08811486, 0.3343631, 0.1388174...","[0.18779568, -0.08811486, 0.3343631, 0.1388174...","[-0.18099773, 0.004607234, -0.16927706, 0.0171...","[-0.18038093, 0.14802821, -0.13782012, 0.14190...","[-0.16235968, 0.17673601, 0.15929715, -0.03040...","[-0.8669583, -0.13546164, -0.1602493, -0.27697...","[-0.5823707, 0.99805766, 0.020104025, -0.13006...","[-0.06712292, -0.36601505, 0.16617914, -0.2152...","[0.15473132, 0.32326397, -0.18749355, 0.166695...","[-0.18629397, 0.18256846, 0.15284514, 0.173631...","[0.25970796, -0.13424258, -0.047555935, 0.1796...","[0.22400653, -0.09790707, 0.1721392, 0.8137901...","[-0.35535458, -0.12494511, -0.06308618, -0.016...","[-0.205078, -0.17295104, 0.037639644, 0.259601...","[0.13397782, -0.7849154, 0.04519084, -0.386736...","[0.18882008, 0.13763928, -0.23123129, -0.26621...","[-0.24629422, 0.124695316, -0.08914735, -0.329...","[-0.14544696, 0.120337486, 0.7013563, 0.282824...","[-0.23398966, 0.3831126, 0.07868944, 0.1638495...","[0.09448146, 0.06698276, 0.04008036, 0.3741016...","[0.87966245, -0.17448163, 0.116820045, -0.1264...","[0.089285225, -0.19105718, 0.11147936, -0.1877...","[0.23618549, -0.7071938, 0.18400525, -0.120960...","[0.034290463, -0.10741352, -0.17980522, -0.318...","[0.013804657, -0.30630273, -0.09166447, -0.211...","[-0.39416385, -0.7935572, -0.13475567, -0.0088...","[0.16949327, -0.23509161, 0.061761376, -0.1377...","[0.7256753, -0.18361443, -0.0339464, 0.0309750...","[-0.16343386, 0.18606912, -0.14501391, -0.1812...","[0.26688963, 0.19591579, 0.17756972, 0.1942677...","[-0.044759233, 0.16857605, 0.13786303, -0.0555...","[0.09942518, -0.90387946, -0.09536408, -0.0807...","[0.21912523, -0.17620154, 0.17831454, -0.14579..."
32,a138dca7063cc5e1b85b91d8ab125f723bf9fe980cb9fb...,2022-01-09 03:47:54.564009,"[0.030128572, -0.14366034, 0.23352952, -0.0620...","[0.030128572, -0.14366034, 0.23352952, -0.0620...","[-0.13098614, 0.36683565, -0.047764484, 0.1031...","[-0.06576882, 0.035876427, -0.1374832, 0.18058...","[-0.062203724, 0.013252216, 0.101086065, -0.32...","[-0.9467342, -0.16773951, -0.31774285, -0.2120...","[-0.4041704, 0.9998826, 0.2983847, -0.09761761...","[-0.1043461, -0.31871954, 0.37439665, -0.39395...","[0.11339976, 0.3516426, -0.19963801, 0.0792364...","[-0.17262678, 0.066424474, 0.14512773, 0.30068...","[0.23064466, -0.15577152, -0.19447748, 0.00897...","[0.2050889, -0.35381114, 0.2905312, 0.98989177...","[-0.39619386, -0.123395965, -0.055173505, -0.0...","[-0.19496576, -0.11473216, -0.07276453, 0.3893...","[0.08981846, -0.689694, 0.31830633, -0.3305799...","[0.1926142, 0.11935668, -0.32281294, -0.352668...","[-0.3646234, 0.19423604, -0.006658244, -0.2835...","[-0.1958541, 0.008144188, 0.7041161, 0.2838730...","[-0.2752991, 0.22320299, 0.17659031, 0.0315162...","[0.1841749, 0.16387758, 0.018431861, 0.3605761...","[0.9985117, -0.041222397, 0.1051562, -0.059861...","[0.29702538, -0.3399145, 0.17574702, -0.141670...","[0.111098245, -0.7943108, 0.19219264, -0.03527...","[0.10905756, -0.061749835, -0.35204086, -0.299...","[0.07786302, -0.34731367, -0.19024737, -0.3859...","[-0.33233565, -0.66132575, -0.07777117, -0.162...","[0.1961683, -0.2126411, 0.19061258, -0.1765806...","[0.964074, -0.01082817, -0.09754, 0.059338946,...","[-0.19465396, 0.13651894, -0.07136886, -0.1018...","[0.20432155, -0.19470443, 0.16006067, 0.119346...","[-0.33648184, 0.19318512, 0.12547617, -0.05148...","[0.36984462, -0.9330921, -0.028417695, -0.1352...","[0.33848855, -0.3356167, -0.0029471994, -0.105..."
5

In [374]:
columns_part

['1_part_emb',
 '2_part_emb',
 '3_part_emb',
 '4_part_emb',
 '5_part_emb',
 '6_part_emb',
 '7_part_emb',
 '8_part_emb',
 '9_part_emb',
 '10_part_emb',
 '11_part_emb',
 '12_part_emb',
 '13_part_emb',
 '14_part_emb',
 '15_part_emb',
 '16_part_emb',
 '17_part_emb',
 '18_part_emb',
 '19_part_emb',
 '20_part_emb',
 '21_part_emb',
 '22_part_emb',
 '23_part_emb',
 '24_part_emb',
 '25_part_emb',
 '26_part_emb',
 '27_part_emb',
 '28_part_emb',
 '29_part_emb',
 '30_part_emb',
 '31_part_emb',
 '32_part_emb']

In [384]:
# data['min_1_part_emb'] = data['1_part_emb'].apply(lambda x: np.min(x, axis=0))
# data

In [385]:
data = tmp_df
for col in columns_part:
    print(col)
    data[f'min_{col}'] = data[col].apply(lambda x: np.min(x, axis=0))
    data[f'max_{col}'] = data[col].apply(lambda x: np.max(x, axis=0))
    data[f'sum_{col}'] = data[col].apply(lambda x: np.sum(x, axis=0))
    data[f'std_{col}'] = data[col].apply(lambda x: np.std(x, axis=0))
    data[f'mean_{col}'] = data[col].apply(lambda x: np.mean(x, axis=0))

1_part_emb
2_part_emb
3_part_emb
4_part_emb
5_part_emb
6_part_emb
7_part_emb
8_part_emb
9_part_emb
10_part_emb
11_part_emb
12_part_emb
13_part_emb
14_part_emb
15_part_emb
16_part_emb
17_part_emb
18_part_emb
19_part_emb
20_part_emb
21_part_emb
22_part_emb
23_part_emb
24_part_emb
25_part_emb
26_part_emb
27_part_emb
28_part_emb
29_part_emb
30_part_emb
31_part_emb
32_part_emb


In [386]:
data

,client_id,event_time,embedding,1_part_emb,2_part_emb,3_part_emb,4_part_emb,5_part_emb,6_part_emb,7_part_emb,8_part_emb,9_part_emb,10_part_emb,11_part_emb,12_part_emb,13_part_emb,14_part_emb,15_part_emb,16_part_emb,17_part_emb,18_part_emb,19_part_emb,20_part_emb,21_part_emb,22_part_emb,23_part_emb,24_part_emb,25_part_emb,26_part_emb,27_part_emb,28_part_emb,29_part_emb,30_part_emb,31_part_emb,32_part_emb,min_1_part_emb,max_1_part_emb,sum_1_part_emb,std_1_part_emb,mean_1_part_emb,min_2_part_emb,max_2_part_emb,sum_2_part_emb,std_2_part_emb,mean_2_part_emb,min_3_part_emb,max_3_part_emb,sum_3_part_emb,std_3_part_emb,mean_3_part_emb,min_4_part_emb,max_4_part_emb,sum_4_part_emb,std_4_part_emb,mean_4_part_emb,min_5_part_emb,max_5_part_emb,sum_5_part_emb,std_5_part_emb,mean_5_part_emb,min_6_part_emb,max_6_part_emb,sum_6_part_emb,std_6_part_emb,mean_6_part_emb,min_7_part_emb,max_7_part_emb,sum_7_part_emb,std_7_part_emb,mean_7_part_emb,min_8_part_emb,max_8_part_emb,sum_8_part_emb,std_8_part_emb,mean_8_part_emb,min_9_part_emb,max_9_part_emb,sum_9_part_emb,std_9_part_emb,mean_9_part_emb,min_10_part_emb,max_10_part_emb,sum_10_part_emb,std_10_part_emb,mean_10_part_emb,min_11_part_emb,max_11_part_emb,sum_11_part_emb,std_11_part_emb,mean_11_part_emb,min_12_part_emb,max_12_part_emb,sum_12_part_emb,std_12_part_emb,mean_12_part_emb,min_13_part_emb,max_13_part_emb,sum_13_part_emb,std_13_part_emb,mean_13_part_emb,min_14_part_emb,max_14_part_emb,sum_14_part_emb,std_14_part_emb,mean_14_part_emb,min_15_part_emb,max_15_part_emb,sum_15_part_emb,std_15_part_emb,mean_15_part_emb,min_16_part_emb,max_16_part_emb,sum_16_part_emb,std_16_part_emb,mean_16_part_emb,min_17_part_emb,max_17_part_emb,sum_17_part_emb,std_17_part_emb,mean_17_part_emb,min_18_part_emb,max_18_part_emb,sum_18_part_emb,std_18_part_emb,mean_18_part_emb,min_19_part_emb,max_19_part_emb,sum_19_part_emb,std_19_part_emb,mean_19_part_emb,min_20_part_emb,max_20_part_emb,sum_20_part_emb,std_20_part_emb,mean_20_part_emb,min_21_part_emb,max_21_part_emb,sum_21_part_emb,std_21_part_emb,mean_21_part_emb,min_22_part_emb,max_22_part_emb,sum_22_part_emb,std_22_part_emb,mean_22_part_emb,min_23_part_emb,max_23_part_emb,sum_23_part_emb,std_23_part_emb,mean_23_part_emb,min_24_part_emb,max_24_part_emb,sum_24_part_emb,std_24_part_emb,mean_24_part_emb,min_25_part_emb,max_25_part_emb,sum_25_part_emb,std_25_part_emb,mean_25_part_emb,min_26_part_emb,max_26_part_emb,sum_26_part_emb,std_26_part_emb,mean_26_part_emb,min_27_part_emb,max_27_part_emb,sum_27_part_emb,std_27_part_emb,mean_27_part_emb,min_28_part_emb,max_28_part_emb,sum_28_part_emb,std_28_part_emb,mean_28_part_emb,min_29_part_emb,max_29_part_emb,sum_29_part_emb,std_29_part_emb,mean_29_part_emb,min_30_part_emb,max_30_part_emb,sum_30_part_emb,std_30_part_emb,mean_30_part_emb,min_31_part_emb,max_31_part_emb,sum_31_part_emb,std_31_part_emb,mean_31_part_emb,min_32_part_emb,max_32_part_emb,sum_32_part_emb,std_32_part_emb,mean_32_part_emb
28,a138dca7063cc5e1b85b91d8ab125f723bf9fe980cb9fb...,2022-01-06 03:41:47.104645,"[0.18779568, -0.08811486, 0.3343631, 0.1388174...","[0.18779568, -0.08811486, 0.3343631, 0.1388174...","[-0.18099773, 0.004607234, -0.16927706, 0.0171...","[-0.18038093, 0.14802821, -0.13782012, 0.14190...","[-0.16235968, 0.17673601, 0.15929715, -0.03040...","[-0.8669583, -0.13546164, -0.1602493, -0.27697...","[-0.5823707, 0.99805766, 0.020104025, -0.13006...","[-0.06712292, -0.36601505, 0.16617914, -0.2152...","[0.15473132, 0.32326397, -0.18749355, 0.166695...","[-0.18629397, 0.18256846, 0.15284514, 0.173631...","[0.25970796, -0.13424258, -0.047555935, 0.1796...","[0.22400653, -0.09790707, 0.1721392, 0.8137901...","[-0.35535458, -0.12494511, -0.06308618, -0.016...","[-0.205078, -0.17295104, 0.037639644, 0.259601...","[0.13397782, -0.7849154, 0.04519084, -0.386736...","[0.18882008, 0.13763928, -0.23123129, -0.26621...","[-0.24629422, 0.124695316, -0.08914735, -0.329...","[-0.14544696, 0.120337486, 0.7013563, 0.282824...","[-0.23398966, 0.3831126, 0.07

In [367]:
np.array([[1,2],[2,3]])

array([[1, 2],
       [2, 3]])

In [368]:
tmp_df

,client_id,event_time,embedding,1_part_emb,2_part_emb,3_part_emb,4_part_emb,5_part_emb,6_part_emb,7_part_emb,8_part_emb,9_part_emb,10_part_emb,11_part_emb,12_part_emb,13_part_emb,14_part_emb,15_part_emb,16_part_emb,17_part_emb,18_part_emb,19_part_emb,20_part_emb,21_part_emb,22_part_emb,23_part_emb,24_part_emb,25_part_emb,26_part_emb,27_part_emb,28_part_emb,29_part_emb,30_part_emb,31_part_emb,32_part_emb
28,a138dca7063cc5e1b85b91d8ab125f723bf9fe980cb9fb...,2022-01-06 03:41:47.104645,"[0.18779568, -0.08811486, 0.3343631, 0.1388174...","[0.18779568, -0.08811486, 0.3343631, 0.1388174...","[-0.18099773, 0.004607234, -0.16927706, 0.0171...","[-0.18038093, 0.14802821, -0.13782012, 0.14190...","[-0.16235968, 0.17673601, 0.15929715, -0.03040...","[-0.8669583, -0.13546164, -0.1602493, -0.27697...","[-0.5823707, 0.99805766, 0.020104025, -0.13006...","[-0.06712292, -0.36601505, 0.16617914, -0.2152...","[0.15473132, 0.32326397, -0.18749355, 0.166695...","[-0.18629397, 0.18256846, 0.15284514, 0.173631...","[0.25970796, -0.13424258, -0.047555935, 0.1796...","[0.22400653, -0.09790707, 0.1721392, 0.8137901...","[-0.35535458, -0.12494511, -0.06308618, -0.016...","[-0.205078, -0.17295104, 0.037639644, 0.259601...","[0.13397782, -0.7849154, 0.04519084, -0.386736...","[0.18882008, 0.13763928, -0.23123129, -0.26621...","[-0.24629422, 0.124695316, -0.08914735, -0.329...","[-0.14544696, 0.120337486, 0.7013563, 0.282824...","[-0.23398966, 0.3831126, 0.07868944, 0.1638495...","[0.09448146, 0.06698276, 0.04008036, 0.3741016...","[0.87966245, -0.17448163, 0.116820045, -0.1264...","[0.089285225, -0.19105718, 0.11147936, -0.1877...","[0.23618549, -0.7071938, 0.18400525, -0.120960...","[0.034290463, -0.10741352, -0.17980522, -0.318...","[0.013804657, -0.30630273, -0.09166447, -0.211...","[-0.39416385, -0.7935572, -0.13475567, -0.0088...","[0.16949327, -0.23509161, 0.061761376, -0.1377...","[0.7256753, -0.18361443, -0.0339464, 0.0309750...","[-0.16343386, 0.18606912, -0.14501391, -0.1812...","[0.26688963, 0.19591579, 0.17756972, 0.1942677...","[-0.044759233, 0.16857605, 0.13786303, -0.0555...","[0.09942518, -0.90387946, -0.09536408, -0.0807...","[0.21912523, -0.17620154, 0.17831454, -0.14579..."
32,a138dca7063cc5e1b85b91d8ab125f723bf9fe980cb9fb...,2022-01-09 03:47:54.564009,"[0.030128572, -0.14366034, 0.23352952, -0.0620...","[0.030128572, -0.14366034, 0.23352952, -0.0620...","[-0.13098614, 0.36683565, -0.047764484, 0.1031...","[-0.06576882, 0.035876427, -0.1374832, 0.18058...","[-0.062203724, 0.013252216, 0.101086065, -0.32...","[-0.9467342, -0.16773951, -0.31774285, -0.2120...","[-0.4041704, 0.9998826, 0.2983847, -0.09761761...","[-0.1043461, -0.31871954, 0.37439665, -0.39395...","[0.11339976, 0.3516426, -0.19963801, 0.0792364...","[-0.17262678, 0.066424474, 0.14512773, 0.30068...","[0.23064466, -0.15577152, -0.19447748, 0.00897...","[0.2050889, -0.35381114, 0.2905312, 0.98989177...","[-0.39619386, -0.123395965, -0.055173505, -0.0...","[-0.19496576, -0.11473216, -0.07276453, 0.3893...","[0.08981846, -0.689694, 0.31830633, -0.3305799...","[0.1926142, 0.11935668, -0.32281294, -0.352668...","[-0.3646234, 0.19423604, -0.006658244, -0.2835...","[-0.1958541, 0.008144188, 0.7041161, 0.2838730...","[-0.2752991, 0.22320299, 0.17659031, 0.0315162...","[0.1841749, 0.16387758, 0.018431861, 0.3605761...","[0.9985117, -0.041222397, 0.1051562, -0.059861...","[0.29702538, -0.3399145, 0.17574702, -0.141670...","[0.111098245, -0.7943108, 0.19219264, -0.03527...","[0.10905756, -0.061749835, -0.35204086, -0.299...","[0.07786302, -0.34731367, -0.19024737, -0.3859...","[-0.33233565, -0.66132575, -0.07777117, -0.162...","[0.1961683, -0.2126411, 0.19061258, -0.1765806...","[0.964074, -0.01082817, -0.09754, 0.059338946,...","[-0.19465396, 0.13651894, -0.07136886, -0.1018...","[0.20432155, -0.19470443, 0.16006067, 0.119346...","[-0.33648184, 0.19318512, 0.12547617, -0.05148...","[0.36984462, -0.9330921, -0.028417695, -0.1352...","[0.33848855, -0.3356167, -0.0029471994, -0.105..."
5

In [309]:
stats_dialogs

,sum_pos_current_acc_emb,sum_neg_current_acc_emb,sum_posneg_current_acc_emb,sum_posneg_mean_current_acc_emb,sum_pos_deposits_emb,sum_neg_deposits_emb,sum_posneg_deposits_emb,sum_posneg_mean_deposits_emb,sum_pos_credits_emb,sum_neg_credits_emb,sum_posneg_credits_emb,sum_posneg_mean_credits_emb,sum_pos_credit_cards_emb,sum_neg_credit_cards_emb,sum_posneg_credit_cards_emb,sum_posneg_mean_credit_cards_emb,sum_pos_debit_cards_emb,sum_neg_debit_cards_emb,sum_posneg_debit_cards_emb,sum_posneg_mean_debit_cards_emb,sum_pos_payments_emb,sum_neg_payments_emb,sum_posneg_payments_emb,sum_posneg_mean_payments_emb,sum_pos_investments_emb,sum_neg_investments_emb,sum_posneg_investments_emb,sum_posneg_mean_investments_emb,sum_pos_garant_emb,sum_neg_garant_emb,sum_posneg_garant_emb,sum_posneg_mean_garant_emb,sum_pos_colect_serv_emb,sum_neg_colect_serv_emb,sum_posneg_colect_serv_emb,sum_posneg_mean_colect_serv_emb,sum_pos_salary_emb,sum_neg_salary_emb,sum_posneg_salary_emb,sum_posneg_mean_salary_emb,sum_pos_bank_app_emb,sum_neg_bank_app_emb,sum_posneg_bank_app_emb,sum_posneg_mean_bank_app_emb,sum_pos_car_loans_emb,sum_neg_car_loans_emb,sum_posneg_car_loans_emb,sum_posneg_mean_car_loans_emb,sum_pos_cashback_emb,sum_neg_cashback_emb,sum_posneg_cashback_emb,sum_posneg_mean_cashback_emb,sum_positive_emb,sum_negative_emb,mean_pos_current_acc_emb,mean_neg_current_acc_emb,mean_posneg_current_acc_emb,mean_posneg_mean_current_acc_emb,mean_pos_deposits_emb,mean_neg_deposits_emb,mean_posneg_deposits_emb,mean_posneg_mean_deposits_emb,mean_pos_credits_emb,mean_neg_credits_emb,mean_posneg_credits_emb,mean_posneg_mean_credits_emb,mean_pos_credit_cards_emb,mean_neg_credit_cards_emb,mean_posneg_credit_cards_emb,mean_posneg_mean_credit_cards_emb,mean_pos_debit_cards_emb,mean_neg_debit_cards_emb,mean_posneg_debit_cards_emb,mean_posneg_mean_debit_cards_emb,mean_pos_payments_emb,mean_neg_payments_emb,mean_posneg_payments_emb,mean_posneg_mean_payments_emb,mean_pos_investments_emb,mean_neg_investments_emb,mean_posneg_investments_emb,mean_posneg_mean_investments_emb,mean_pos_garant_emb,mean_neg_garant_emb,mean_posneg_garant_emb,mean_posneg_mean_garant_emb,mean_pos_colect_serv_emb,mean_neg_colect_serv_emb,mean_posneg_colect_serv_emb,mean_posneg_mean_colect_serv_emb,mean_pos_salary_emb,mean_neg_salary_emb,mean_posneg_salary_emb,mean_posneg_mean_salary_emb,mean_pos_bank_app_emb,mean_neg_bank_app_emb,mean_posneg_bank_app_emb,mean_posneg_mean_bank_app_emb,mean_pos_car_loans_emb,mean_neg_car_loans_emb,mean_posneg_car_loans_emb,mean_posneg_mean_car_loans_emb,mean_pos_cashback_emb,mean_neg_cashback_emb,mean_posneg_cashback_emb,mean_posneg_mean_cashback_emb,mean_positive_emb,mean_negative_emb,min_pos_current_acc_emb,min_neg_current_acc_emb,min_posneg_current_acc_emb,min_posneg_mean_current_acc_emb,min_pos_deposits_emb,min_neg_deposits_emb,min_posneg_deposits_emb,min_posneg_mean_deposits_emb,min_pos_credits_emb,min_neg_credits_emb,min_posneg_credits_emb,min_posneg_mean_credits_emb,min_pos_credit_cards_emb,min_neg_credit_cards_emb,min_posneg_credit_cards_emb,min_posneg_mean_credit_cards_emb,min_pos_debit_cards_emb,min_neg_debit_cards_emb,min_posneg_debit_cards_emb,min_posneg_mean_debit_cards_emb,min_pos_payments_emb,min_neg_payments_emb,min_posneg_payments_emb,min_posneg_mean_payments_emb,min_pos_investments_emb,min_neg_investments_emb,min_posneg_investments_emb,min_posneg_mean_investments_emb,min_pos_garant_emb,min_neg_garant_emb,min_posneg_garant_emb,min_posneg_mean_garant_emb,min_pos_colect_serv_emb,min_neg_colect_serv_emb,min_posneg_colect_serv_emb,min_posneg_mean_colect_serv_emb,min_pos_salary_emb,min_neg_salary_emb,min_posneg_salary_emb,min_posneg_mean_salary_emb,min_pos_bank_app_emb,min_neg_bank_app_emb,min_posneg_bank_app_emb,min_posneg_mean_bank_app_emb,min_pos_car_loans_emb,min_neg_car_loans_emb,min_posneg_car_loans_emb,min_posneg_mean_car_loans_emb,min_pos_cashback_emb,min_neg_cashback_emb,min_posneg_cashback_emb,min_posneg_mean_cashback_emb,min_positive_emb,min_negativ

In [293]:
# all_dialogs_df

In [103]:
# tmp_dialogs = all_dialogs_df[:1]
# tmp_emb = tmp_dialogs['embedding'].item()
# len(tmp_emb)
tmp_dialogs_df = all_dialogs_df['embedding']

# list(tmp_dialogs_df['embedding'].values)
# [None, ...]
# tmp_dialogs_df['embedding'].values
# tmp_dialogs_df.to_numpy()
# tmp_dialogs_df.values.flatten()
# tmp_dialogs_df.apply(np.array)
# tmp_dialogs_df.apply(np.array)
embeds = np.vstack(tmp_dialogs_df.apply(np.array).values)
embeds.shape

(1501735, 768)

In [150]:
# Загружаем модель
# model = SentenceTransformer('intfloat/multilingual-e5-large') #0.085 8min 0.075
# model = SentenceTransformer('intfloat/multilingual-e5-small') #
# model = SentenceTransformer('intfloat/multilingual-e5-large-instruct') #
# model = SentenceTransformer('intfloat/e5-mistral-7b-instruct') #
# model = SentenceTransformer('intfloat/multilingual-e5-large') #


# model_dima = SentenceTransformer('DimkKozhemyako/dummy-model') #
# intfloat/e5-small-v2
# intfloat/e5-mistral-7b-instruct
# multilingual-e5-large
# force_download=True,
# embeddings_KSR = bertmodel.encode(stroy_model.ksr_df['Наименование'].values, convert_to_tensor=True)
# embeddings_KSR = bertmodel.encode(stroy_model.ksr_df['Наименование'].values, convert_to_tensor=True)
query= "Здравствуйте! Прошу выкупить у меня облигации Дэни Колл 21.02.2020 согласно дополнительной оферте по номинальной стоимости 1000 рублей в полном объёме портфеля! Прошу подтвердить принятие поручения"
query = "Какие программы ипотечного кредитования предлагает ваш банк, и какие требования необходимо выполнить для их получения?"
query = "Какой размер первоначального взноса требуется, и каковы процентные ставки по различным программам?"
query = """
Оператор: Здравствуйте! Чем мы можем вам помочь в отношении нашего банковского приложения?
Клиент: Добрый день! У меня возникли некоторые вопросы по использованию вашего приложения. Я недавно установил его, но есть некоторые функции, которые мне не совсем понятны.
Оператор: Хорошо, мы с радостью разъясним все непонятные моменты. Какие именно функции приложения вызывают у вас затруднения?
Клиент: Во-первых, я не могу разобраться, как правильно настроить push-уведомления о новых операциях по счету. Кажется, я включил их, но уведомления так и не приходят.
Оператор: Понял вашу проблему. Для настройки push-уведомлений нужно перейти в раздел "Настройки" приложения, затем выбрать "Уведомления" и активировать опцию "Уведомления о новых операциях". После этого вы должны начать получать уведомления.
Клиент: Спасибо, я попробую сделать это. Еще один вопрос – как мне перевести деньги на другой счет внутри вашего банка через приложение?
"""

query = """
Клиент: Здравствуйте, я хотел бы подключить услугу мобильного банка.
Оператор: Хорошо, для подключения мобильного банка необходимо иметь доступ к интернет-банку. У вас уже есть подключенный интернет-банк?
Клиент: Да, у меня есть интернет-банк.
Оператор: Отлично, тогда вы можете подключить мобильное приложение прямо через интернет-банк. Вам нужно зайти в раздел "Управление услугами" и выбрать "Подключить мобильный банк". После этого вам придет SMS с кодом подтверждения.
Клиент: Понятно, спасибо за информацию.
"""

query = """
Клиент: Здравствуйте, я хотел бы узнать, можно ли оплачивать кредит через ваше мобильное приложение?
Оператор: Да, конечно. У нас есть удобное мобильное приложение, которое позволяет производить платежи по кредитам в любое время и в любом месте.
Клиент: Отлично, а как именно это делается?
Оператор: 1) Вы скачиваете наше мобильное приложение из App Store или Google Play.
2) Проходите процесс регистрации и авторизации.
3) В разделе "Платежи" выбираете пункт "Кредит".
4) Вводите необходимую сумму и реквизиты кредита.
5) Подтверждаете платеж.
Клиент: Понятно, спасибо за разъяснение. Это очень удобно.
"""

query = """
Клиент: Добрый день, скажите, пожалуйста, можно ли открыть вклад через систему интернет-банкинга вашего банка?
Оператор: Да, конечно. Наш интернет-банкинг предоставляет такую возможность.
Клиент: Расскажите, пожалуйста, как это сделать?
Оператор: 1) Вы входите в систему интернет-банкинга.
2) В разделе "Вклады" выбираете нужный вид вклада.
3) Заполняете необходимую информацию и подписываете договор электронной подписью.
"""

query = "курсы валют доллары инвестиции"
query = "машину Автокредиты первоначальный взнос"
query = "Повышенные ставки по вкладам, кешбэк до 10% бонусами СберСпасибо, до 6 проходов в бизнес-залы"
query = "Получайте минимальный набор привилегий, если храните от 2 млн руб или оплачивайте картами Сбера от 150 тыс. руб в месяц. Если не выполнять условия, действие привилегий приостановится или их станет меньше"

emb = bertmodel.encode(query)
len(emb)


768

In [170]:
sample_query = ["Здравствуйте! Прошу выкупить у меня облигации Дэни Колл 21.02.2020 согласно дополнительной оферте по номинальной стоимости 1000 рублей в полном объёме портфеля! Прошу подтвердить принятие поручения",
"Какие программы ипотечного кредитования предлагает ваш банк, и какие требования необходимо выполнить для их получения?",
"Какой размер первоначального взноса требуется, и каковы процентные ставки по различным программам?",
"""
Оператор: Здравствуйте! Чем мы можем вам помочь в отношении нашего банковского приложения?
Клиент: Добрый день! У меня возникли некоторые вопросы по использованию вашего приложения. Я недавно установил его, но есть некоторые функции, которые мне не совсем понятны.
Оператор: Хорошо, мы с радостью разъясним все непонятные моменты. Какие именно функции приложения вызывают у вас затруднения?
Клиент: Во-первых, я не могу разобраться, как правильно настроить push-уведомления о новых операциях по счету. Кажется, я включил их, но уведомления так и не приходят.
Оператор: Понял вашу проблему. Для настройки push-уведомлений нужно перейти в раздел "Настройки" приложения, затем выбрать "Уведомления" и активировать опцию "Уведомления о новых операциях". После этого вы должны начать получать уведомления.
Клиент: Спасибо, я попробую сделать это. Еще один вопрос – как мне перевести деньги на другой счет внутри вашего банка через приложение?
""",

"""
Клиент: Здравствуйте, я хотел бы подключить услугу мобильного банка.
Оператор: Хорошо, для подключения мобильного банка необходимо иметь доступ к интернет-банку. У вас уже есть подключенный интернет-банк?
Клиент: Да, у меня есть интернет-банк.
Оператор: Отлично, тогда вы можете подключить мобильное приложение прямо через интернет-банк. Вам нужно зайти в раздел "Управление услугами" и выбрать "Подключить мобильный банк". После этого вам придет SMS с кодом подтверждения.
Клиент: Понятно, спасибо за информацию.
""",

"""
Клиент: Здравствуйте, я хотел бы узнать, можно ли оплачивать кредит через ваше мобильное приложение?
Оператор: Да, конечно. У нас есть удобное мобильное приложение, которое позволяет производить платежи по кредитам в любое время и в любом месте.
Клиент: Отлично, а как именно это делается?
Оператор: 1) Вы скачиваете наше мобильное приложение из App Store или Google Play.
2) Проходите процесс регистрации и авторизации.
3) В разделе "Платежи" выбираете пункт "Кредит".
4) Вводите необходимую сумму и реквизиты кредита.
5) Подтверждаете платеж.
Клиент: Понятно, спасибо за разъяснение. Это очень удобно.
""",

"""
Клиент: Добрый день, скажите, пожалуйста, можно ли открыть вклад через систему интернет-банкинга вашего банка?
Оператор: Да, конечно. Наш интернет-банкинг предоставляет такую возможность.
Клиент: Расскажите, пожалуйста, как это сделать?
Оператор: 1) Вы входите в систему интернет-банкинга.
2) В разделе "Вклады" выбираете нужный вид вклада.
3) Заполняете необходимую информацию и подписываете договор электронной подписью.
""",

"курсы валют доллары инвестиции",
"машину Автокредиты первоначальный взнос",
"Повышенные ставки по вкладам, кешбэк до 10% бонусами СберСпасибо, до 6 проходов в бизнес-залы",
"Получайте минимальный набор привилегий, если храните от 2 млн руб или оплачивайте картами Сбера от 150 тыс. руб в месяц. Если не выполнять условия, действие привилегий приостановится или их станет меньше",
                
"""
Я обращаюсь к вам по вопросу оформления ипотечного кредита. В настоящий момент я рассматриваю возможность приобретения квартиры для моей семьи, и ваш банк является одним из наиболее привлекательных вариантов для получения ипотеки.
Я бы хотел получить подробную информацию об условиях ипотечного кредитования в вашем банке. Меня интересуют следующие вопросы:

Какие программы ипотечного кредитования предлагает ваш банк, и какие требования необходимо выполнить для их получения?
Какой размер первоначального взноса требуется, и каковы процентные ставки по различным программам?
Какие документы необходимо предоставить для рассмотрения заявки на ипотеку?
Возможно ли досрочное погашение ипотеки без штрафных санкций, и на каких условиях это осуществляется?
Какие дополнительные услуги и страховые программы предлагает ваш банк при оформлении ипотеки?                
""",
]

In [191]:
sample_query = [
    
"Какие программы ипотечного кредитования предлагает ваш банк, и какие требования необходимо выполнить для их получения?",
"Какой размер первоначального взноса требуется, и каковы процентные ставки по различным программам?",

"""
Клиент: Здравствуйте, я хотел бы узнать, можно ли оплачивать кредит через ваше мобильное приложение?
Оператор: Да, конечно. У нас есть удобное мобильное приложение, которое позволяет производить платежи по кредитам в любое время и в любом месте.
Клиент: Отлично, а как именно это делается?
Оператор: 1) Вы скачиваете наше мобильное приложение из App Store или Google Play.
2) Проходите процесс регистрации и авторизации.
3) В разделе "Платежи" выбираете пункт "Кредит".
4) Вводите необходимую сумму и реквизиты кредита.
5) Подтверждаете платеж.
Клиент: Понятно, спасибо за разъяснение. Это очень удобно.
""",

"Повышенные ставки по вкладам, кешбэк до 10% бонусами СберСпасибо, до 6 проходов в бизнес-залы",
                
"""
Я обращаюсь к вам по вопросу оформления ипотечного кредита. В настоящий момент я рассматриваю возможность приобретения квартиры для моей семьи, и ваш банк является одним из наиболее привлекательных вариантов для получения ипотеки.
Я бы хотел получить подробную информацию об условиях ипотечного кредитования в вашем банке. Меня интересуют следующие вопросы:

Какие программы ипотечного кредитования предлагает ваш банк, и какие требования необходимо выполнить для их получения?
Какой размер первоначального взноса требуется, и каковы процентные ставки по различным программам?
Какие документы необходимо предоставить для рассмотрения заявки на ипотеку?
Возможно ли досрочное погашение ипотеки без штрафных санкций, и на каких условиях это осуществляется?
Какие дополнительные услуги и страховые программы предлагает ваш банк при оформлении ипотеки?                
""",

    


"""
ему с большим объемам данных решали через сэмплирование. Далее все фичи рассчитывали только для сэмплированного набора данных, что значительно упростило и ускорило процесс подготовки фичей.
В качестве базовых моделей использовали бустинг модели и ???? на базе эмбеддингов pytorch-lifestream
Для бустинг моделей генерировали фичи в виде одного объекта
Для pytorch-lifestream ген
""",

"""
Надо придумать много крутых фичей и желательно описать их. Также интересная тема создать эмбеддинги и векторные представления данных для обучения нейронок
Использование инструментов для командной работы: гугл таблицы, гугл докс, миро, регулярные встречи в Яндекс.Телемост, Гугл Мит.
""",

"""
Подумать про прод: тяжесть модели - будут ли они смотреть, как быстро модель отрабатывает
Использование внешних данных
Контроль лика данных
""",

"""
олжен быть описан EDA, приведены инсайты. Отражены проблемы при обработке и пути их решения.
очистка, заполнение, кодировка, нормализация, мультивариантная импутация


Уменьшение объема, предобработка, EDA ( углубленный анализ данных,, устранение аномалий, агрегация и т.д.)

Поиск и описание инсайтов.Проведение EDA, описание трендов и закономерностей в данных, визуализация результатов.

Также смотри раздел "Предобработка"в в архитеорурек
""",

"""
import pandas as pd
import numpy as np
import torch
import re

from typing import List, Optional

from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings("ignore")

""",
    
    
]

len(sample_query)

10

In [176]:
query_emdes = np.array([bertmodel.encode(q) for q in sample_query])

In [177]:
# tmp_emb
# tmp_dialogs_df['embedding'].values.apply(lambda x: )
# item = tmp_dialogs_df['embedding'].values[0]
# item
# emb[None, ...]

In [178]:
%%time
# cosine_similarity(tmp_emb[None, ...], emb[None, ...])
# cos_sim_res = cosine_similarity(embeds, emb[None, ...])
cos_sim_res = cosine_similarity(embeds, query_emdes)


CPU times: total: 10.2 s
Wall time: 3.62 s


In [193]:
cos_sim_res[122]

array([-0.04990443, -0.03318198, -0.03848876, -0.02861523, -0.04405659,
       -0.03667687, -0.04531683, -0.03228377, -0.04539747,  0.00911165],
      dtype=float32)

In [194]:
cos_sim_res.argmin(axis=1)
cos_sim_res[0]

array([ 0.0017028 ,  0.01390065, -0.00179355,  0.01555294,  0.00229831,
       -0.0049457 , -0.00557397,  0.00443213, -0.00305652,  0.02782652],
      dtype=float32)

In [153]:
cos_sim_res[cos_sim_res.argmin()], cos_sim_res[cos_sim_res.argmax()]

(array([-0.08546399], dtype=float32), array([0.06661562], dtype=float32))

In [112]:
cos_sim_res[cos_sim_res.argmin()], cos_sim_res[cos_sim_res.argmax()]

(array([-0.09697731], dtype=float32), array([0.05809498], dtype=float32))

In [109]:
cos_sim_res[cos_sim_res.argmin()], cos_sim_res[cos_sim_res.argmax()]

(array([-0.12002011], dtype=float32), array([0.0663759], dtype=float32))

In [106]:
cos_sim_res[cos_sim_res.argmin()], cos_sim_res[cos_sim_res.argmax()]

(array([-0.11655792], dtype=float32), array([0.0458304], dtype=float32))

In [98]:
cos_sim_res[cos_sim_res.argmin()], cos_sim_res[cos_sim_res.argmax()]

(array([-0.0657125], dtype=float32), array([0.12628223], dtype=float32))

In [94]:
cos_sim_res[cos_sim_res.argmin()], cos_sim_res[cos_sim_res.argmax()]

(array([-0.0657125], dtype=float32), array([0.10043626], dtype=float32))

In [91]:
cos_sim_res[cos_sim_res.argmin()], cos_sim_res[cos_sim_res.argmax()]

(array([-0.05416185], dtype=float32), array([0.09081855], dtype=float32))

7127

In [26]:
length_embedding = 768
import math
math.ceil(768/30)



25.6

In [18]:
tmp_emb

array([-3.24845104e-03,  1.40231192e-01,  5.89073785e-02,  4.11099941e-01,
        7.39320815e-02,  8.33053738e-02, -1.18855335e-01, -1.67109758e-01,
        5.17857037e-02, -3.03766951e-02,  1.32092804e-01,  1.83840677e-01,
       -7.44446665e-02,  1.27890423e-01, -9.43563282e-02,  7.92352632e-02,
        5.27945533e-02, -2.20563505e-02, -3.62790264e-02,  1.00175418e-01,
       -9.96092796e-01,  8.08785856e-02,  1.71430018e-02,  9.46220756e-02,
        1.68545514e-01, -2.45340429e-02,  8.57777372e-02, -1.51411444e-01,
       -1.17816925e-01,  1.73420474e-01, -3.86762977e-01, -9.85430002e-01,
       -4.96158302e-01, -1.87464774e-01, -4.55329530e-02,  3.02412771e-02,
       -9.39209536e-02, -3.36512141e-02, -1.22009041e-02,  6.11187331e-02,
        1.66868046e-02, -7.97764510e-02,  1.47239238e-01, -3.81429553e-01,
        9.65803638e-02,  1.63161248e-01,  1.00291058e-01, -1.11952357e-01,
        1.84971794e-01, -4.56409678e-02,  9.49985813e-03, -6.17184639e-02,
        1.04066685e-01, -

In [13]:
# all_dialogs_df

In [62]:
def calculate_embedding_stats(data):
    stats = pd.DataFrame()

    # Группируем данные по client_id
    grouped = data.groupby('client_id')['embedding'].apply(lambda x: np.array(x.tolist()))
    # Средний эмбеддинг диалога
    stats['mean_embedding'] = grouped.apply(lambda x: np.mean(x, axis=0))
    
    # Сумма
    stats['sum_embedding'] = grouped.apply(lambda x: np.sum(x))
    
    # Дисперсия эмбеддингов диалога
    stats['embedding_variance'] = grouped.apply(lambda x: np.var(x, axis=0).mean())

    # Минимальное/максимальное расстояние между эмбеддингами
    distances = grouped.apply(lambda x: squareform(pdist(x, 'euclidean')))
    stats['min_embedding_distance'] = distances.apply(np.min)
    stats['max_embedding_distance'] = distances.apply(np.max)

    # Средняя разница между соседними эмбеддингами
    stats['mean_consecutive_distance'] = grouped.apply(lambda x: np.mean([np.linalg.norm(x[i] - x[i-1]) for i in range(1, len(x))]))

    # Энтропия эмбеддингов диалога
    # stats['embedding_entropy'] = grouped.apply(lambda x: np.mean(-np.sum(np.log(np.var(x, axis=0)) * np.var(x, axis=0), axis=1)))
    stats['embedding_entropy'] = grouped.apply(lambda x: -np.sum(np.log(np.var(x, axis=0)) * np.var(x, axis=0)))

    # Длина диалога в эмбеддингах
    stats['embedding_path_length'] = grouped.apply(lambda x: np.sum([np.linalg.norm(x[i] - x[i-1]) for i in range(1, len(x))]))

    # Разложение на главные компоненты (PCA) эмбеддингов диалога
    stats['pca_embedding'] = grouped.apply(lambda x: PCA().fit_transform(x)[0])

    return stats

In [67]:
%%time
# Формируем фичи по таргету
begin_date = datetime(2022, 1, 1, 0, 0, 0)
start_date = datetime(2022, 1, 1, 0, 0, 0)

end_date = datetime(2023, 1, 31, 0, 0, 0)

# Бланк-датафрейм с клиентами 
uniq_clients_df = all_dialogs_df[['client_id']].drop_duplicates()

# Итоговый датасет 
union_client_agg_df = pd.DataFrame()

# Бежим по месяцам и расчитываем статистики для клиента берем предыдущие месяцы
for i in trange(((end_date - start_date).days//30 + 1)):
    end_date = start_date + relativedelta(months=1) - relativedelta(days=1)
    print(f'start: {start_date}, end: {end_date}')    
    
    select_mon_current_df = all_dialogs_df[all_dialogs_df['event_time'].between(start_date, end_date)]
    select_mon_full_df = all_dialogs_df[all_dialogs_df['event_time'].between(begin_date, end_date)]
    print(select_mon_current_df.shape, select_mon_full_df.shape)
    
    # client_agg_df = uniq_clients_df.copy()
    client_agg_df = select_mon_full_df[['client_id']].drop_duplicates().copy()
    report_next_end = start_date + relativedelta(months=2) - relativedelta(days=1)
    client_agg_df['report_next_end'] = report_next_end
    client_agg_df = client_agg_df.set_index('client_id')    
    
    # Считаем статистики только за прошедший месяц
    stats_mon = calculate_embedding_stats(select_mon_current_df)
    # Считаем статистики за весь прошедший период
    stats_fulltime = calculate_embedding_stats(select_mon_full_df)
    
    client_agg_df = client_agg_df.merge(stats_mon, left_index=True, right_index=True, how='left')
    client_agg_df = client_agg_df.merge(stats_fulltime.add_prefix('fullt_'), left_index=True, right_index=True, how='left')
    
    union_client_agg_df = pd.concat([union_client_agg_df, client_agg_df])

    start_date = start_date + relativedelta(months=1)
union_client_agg_df.shape    

  0%|          | 0/14 [00:00<?, ?it/s]

start: 2022-01-01 00:00:00, end: 2022-01-31 00:00:00
(62043, 3) (62043, 3)


  7%|▋         | 1/14 [00:31<06:48, 31.44s/it]

start: 2022-02-01 00:00:00, end: 2022-02-28 00:00:00
(45284, 3) (108996, 3)


 14%|█▍        | 2/14 [01:07<06:47, 33.94s/it]

...

 93%|█████████▎| 13/14 [30:24<03:35, 215.60s/it]

start: 2023-02-01 00:00:00, end: 2023-02-28 00:00:00
(0, 3) (1501720, 3)


100%|██████████| 14/14 [34:19<00:00, 147.07s/it]

CPU times: total: 2h 18min 45s
Wall time: 34min 19s


(3589833, 19)

In [71]:
%%time
# Уменьшение размера датафрейма, для таргетов, транзакцй и для фичей
def series_to_int(col_df:pd.Series):
    """
    Перевод в целочисленные типы
    """
    min_val = col_df.min()
    max_val = col_df.max()
    if min_val >= -128 and max_val <= 127:
        col_df = col_df.astype('int8')
    elif min_val >= -32768 and max_val <= 32767:
        col_df = col_df.astype('int16')
    elif min_val >= -2147483648 and max_val <= 2147483647:
        col_df = col_df.astype('int32')
    else:
        col_df = col_df.astype('int64')
    return col_df

def compression_df(df:pd.DataFrame(), datetime_cols:List[str]=[], category_cols:List[str]=[]):
    """
    Уменьшение размера датафрейма, для таргетов, транзакцй и для фичей
    """
    float64_cols = list(df.select_dtypes(include='float64'))  
    df[float64_cols] = df[float64_cols].astype('float32')
    for col in df.columns:
        if col in category_cols:
            df[col] = df[col].astype('category')
        elif col in datetime_cols:
            if df[col].dtypes == 'object':
                df[col] = pd.to_datetime(df[col])
        # Если колонка содержит числа 
        elif is_integer_dtype(df[col]):
            if df[col].dtypes == 'int8':
                continue
            else:
                df[col] = series_to_int(df[col])
        elif is_float_dtype(df[col]):
            # Возможно ли перевести в число
            if np.array_equal(df[col].fillna(0), df[col].fillna(0).astype(int)):
                df[col] = df[col].fillna(0)
                df[col] = series_to_int(df[col])
    return df
union_client_agg_df = compression_df(union_client_agg_df, 
                            datetime_cols=['report_end' ,'report_next_end'],
                           )
union_client_agg_df.shape

CPU times: total: 4.64 s
Wall time: 4.63 s


(3589833, 19)

In [73]:
%%time
# Сохраняем в файл оптимизированный файл 
union_client_agg_df.to_parquet(PATH_DATASET_OUTPUT + 'client_agg_dialog_09_06_2024.parquet')

CPU times: total: 2min 30s
Wall time: 3min 44s


In [74]:
# %%time
# # Сохраняем в файл оптимизированный файл 
# # union_client_agg_df.to_parquet(PATH_DATASET_OUTPUT + 'client_agg_dialog_09_06_2024.parquet', engine='pyarrow')
# filename = 'sample_client_agg_dialog_09_06_2024'
# compression_options = dict(method='zip', archive_name=f'{filename}.csv')
# union_client_agg_df.to_csv(f'{filename}.zip', compression=compression_options)
# # union_client_agg_df.to_csv(PATH_DATASET_OUTPUT + 'client_agg_dialog_09_06_2024.csv')
# union_client_agg_df.shape

In [77]:
# union_client_agg_df.info()

In [78]:
# union_client_agg_df['report_next_end'].max()

In [79]:
# ('2023-01-31 00:00:00')

In [80]:
# stats#['mean_embedding'] = grouped.apply(lambda x: np.mean(x, axis=0))
# grouped.apply(lambda x: np.mean(x, axis=0))
# grouped.apply(lambda x: np.sum(x))

In [80]:
# stats_fulltime['mean_embedding'] = stats_fulltime['mean_embedding'].fillna([[0.0]])
# stats_fulltime['pca_embedding'] = stats_fulltime['pca_embedding'].fillna([[0.0]])
# ser.apply(lambda x: [np.nan] if pd.isnull(x) else x)
# stats_fulltime = stats_fulltime.fillna(0)
# stats_mon.fillna(0)['pca_embedding']

In [71]:
client_agg_df = uniq_clients_df.copy()
report_next_end = start_date + relativedelta(months=2) - relativedelta(days=1)
client_agg_df['report_next_end'] = report_next_end
client_agg_df = client_agg_df.set_index('client_id')

In [72]:
client_agg_df = client_agg_df.merge(stats_mon, left_index=True, right_index=True, how='left')
client_agg_df = client_agg_df.merge(stats_fulltime.add_prefix('fullt_'), left_index=True, right_index=True, how='left')
client_agg_df.shape

(408228, 17)

In [73]:
client_agg_df

,report_next_end,mean_embedding,embedding_variance,min_embedding_distance,max_embedding_distance,mean_consecutive_distance,embedding_entropy,embedding_path_length,pca_embedding,fullt_mean_embedding,fullt_embedding_variance,fullt_min_embedding_distance,fullt_max_embedding_distance,fullt_mean_consecutive_distance,fullt_embedding_entropy,fullt_embedding_path_length,fullt_pca_embedding
client_id,,,,,,,,,,,,,,,,,
a039ad3b595d4f5b1a318076286b0e64e87e8dd9ab4795538e71ea081e4c320a,2022-06-30,"[-0.003248451, 0.14023119, 0.05890738, 0.41109...",0.0,0.0,0.0,NaN,NaN,0.0,[[0.0]],"[0.2190157, -0.06494743, 0.32718855, -0.148597...",0.060432,0.0,17.350304,8.396,109.767662,25.188,"[[10.957912, 0.71575403, 0.011548987, 5.724484..."
a060e69e9e049ad012a4036c4cff6487bb443ad572107f7e6d9b3e680e66c032,2022-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
a08c690dd972d2066188950605635ff08b6bc753664e1ee589fa30cc94e1e04d,2022-06-30,"[-0.009235081, -0.06971426, 0.021549499, 0.416...",0.0,0.0,0.0,NaN,NaN,0.0,[[0.0]],"[-0.009235081, -0.06971426, 0.021549499, 0.416...",0.000000,0.0,0.000000,NaN,NaN,0.000,[[0.0]]
a0c613eea4033501d04458faf2f007dc5a36b75a2c722fe5628ea7b7c4b0fd14,2022-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
a0d0a1a255502b2ccd59db4dbd905c6679909bab8e79b230874b95ee38cb2f97,2022-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
d1cd76f1e0f739b244bd1cc7ca7af391390b7ac8571bfab6944e7622063af765,2022-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
d5919157e576d981382589845aa68e8decc82892b18e98dffe1d9210cdb75606,2022-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
d656bd71016d893b44b231a1d4d09e1e9a92528978d71ec45974c8685aac5107,2022-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
stats_mon.shape
# stats_fulltime

(51511, 8)

In [27]:
select_mon_current_df.groupby(by='client_id')['embedding'].agg({
    
})

In [28]:
select_mon_current_df

,client_id,event_time,embedding
0,a039ad3b595d4f5b1a318076286b0e64e87e8dd9ab4795...,2022-05-02 12:24:30.565231,"[-0.003248451, 0.14023119, 0.05890738, 0.41109..."
4,a08c690dd972d2066188950605635ff08b6bc753664e1e...,2022-05-02 09:12:22.795170,"[-0.009235081, -0.06971426, 0.021549499, 0.416..."
10,a0fa63370e7dbb1cf599dbdf5a6152e618771a63776be9...,2022-05-30 09:06:53.537799,"[-0.1669945, 0.13709365, 0.18547802, 0.5901658..."
47,a2fb6fd0cd3a7af5aa057b0a206a81732e116fae6c7e0a...,2022-05-29 06:44:37.449123,"[0.5710932, -0.38479567, 0.54263395, -0.429531..."
48,a2fb6fd0cd3a7af5aa057b0a206a81732e116fae6c7e0a...,2022-05-09 07:17:04.940099,"[0.16744137, 0.1329933, 0.16055582, 0.38802645..."
...,...,...,...
84761,d912f568f7dc878abf4a9996133bf13798053da8e0da66...,2022-05-18 06:22:58.062891,"[0.032440223, 0.09860634, 0.1994212, 0.2984210..."
84763,d912f568f7dc878abf4a9996133bf13798053da8e0da66...,2022-05-21 10:13:29.854521,"[0.5674321, -0.3097128, 0.7594713, -0.43747067..."
84778,052759e74e402620bb9a42a5af1d64bb996d7baac0e1cd...,2022-05-24 07:47:01.243610,"[0.25777823, -0.04035125, 0.45251316, -0.20103..."
84780,17ddd67fb63a84b50f7d5fffc9859276a44e67c235c900...,2022-05-19 12:53:32.226343,"[0.4742465, -0.1554204, 0.43083036, -0.2594006..."


In [56]:
%%time



stats_df = calculate_embedding_stats(select_mon_current_df)

CPU times: total: 1min 17s
Wall time: 17.1 s


In [ ]:
stats_df

In [40]:
data = select_mon_current_df.copy()
data.shape

(73876, 3)

In [48]:
pd.DataFrame(grouped).reset_index()

,client_id,embedding
0,00026372ff1da433dccb75b7eeb25e363175b84d5bb9e2...,"[[0.59297043, -0.21068758, 0.7144083, -0.41182..."
1,0002b5509d12d4abd6e08359eb19d4ee358063e24b7681...,"[[0.37256125, -0.242806, 0.55156916, -0.451818..."
2,0006218092a640182750857c9c962165053d083d1ca083...,"[[0.07011423, -0.1503446, 0.26275513, -0.05080..."
3,00077a8b8206d6c0cbb886463acb6e9901f54309e148b2...,"[[0.7412198, -0.43468043, 0.7145492, -0.789908..."
4,0008bf239c5af133171e466dd774f9e09a833b7abc6660...,"[[0.30941036, -0.37311456, 0.5001318, -0.52664..."
...,...,...
51506,fff06921f7f1f6b8bab06a4de4bf92775321441ee3dba6...,"[[0.55538344, -0.25558403, 0.7128786, -0.66110..."
51507,fff0ac5010bfbda076eda2eaab44bef3b00abd71ae7a1e...,"[[0.27794403, -0.38389373, 0.5009531, -0.55539..."
51508,fff7abe4759ae512b11cac353f8734866658e97bb425c7...,"[[0.26875737, -0.039456215, 0.45330313, -0.362..."
51509,fffb4dfad27856f24b9487f76cd9dac40210eea3de58e1...,"[[0.4541814, -0.33278686, 0.7704116, -0.474047..."


In [ ]:
# Темпоральные признаки
stats['mean_time_between_events'] = grouped.apply(lambda x: np.mean([data.loc[data['client_id'] == client_id, 'event_time'].diff().dt.total_seconds()[i] for i in range(1, len(x))]))


In [52]:
# Темпоральные признаки
# stats['mean_time_between_events'] = grouped.apply(lambda x, client_id: np.mean([data.loc[data['client_id'] == client_id, 'event_time'].diff().dt.total_seconds()[i] for i in range(1, len(x))]), client_id=data['client_id'])
pd.DataFrame(grouped).reset_index().apply(lambda x: np.mean([data.loc[data['client_id'] == x.client_id, 'event_time'].diff().dt.total_seconds()[i] for i in range(1, len(x.embedding))]), x.client_id=data['client_id'], axis=1)

SyntaxError: expression cannot contain assignment, perhaps you meant "=="? (191500505.py, line 3)

In [53]:
# Темпоральные признаки
stats['mean_time_between_events'] = grouped.apply(lambda x, client_id: np.mean([data.loc[data['client_id'] == client_id, 'event_time'].diff().dt.total_seconds()[i] for i in range(1, len(x))]), client_id=data['client_id'])

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (4,) + inhomogeneous part.

In [ ]:
Все равно в строчке:
stats['mean_time_between_events'] = grouped.apply(lambda x, client_id: np.mean([data.loc[data['client_id'] == client_id, 'event_time'].diff().dt.total_seconds()[i] for i in range(1, len(x))]), client_id=data['client_id'])

Возникает ошибка:
setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (4,) + inhomogeneous part.

In [55]:
%%time
# Темпоральные признаки
def get_mean_time_between_events(group):
    return np.mean(data[data['client_id'] == group.name]['event_time'].diff().dt.total_seconds()[1:])

stats['mean_time_between_events'] = data.groupby('client_id').progress_apply(get_mean_time_between_events)


KeyboardInterrupt: 

In [17]:
all_dialogs_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1501735 entries, 0 to 84983
Data columns (total 3 columns):
 #   Column      Non-Null Count    Dtype         
---  ------      --------------    -----         
 0   client_id   1501735 non-null  object        
 1   event_time  1501735 non-null  datetime64[us]
 2   embedding   1501735 non-null  object        
dtypes: datetime64[us](1), object(2)
memory usage: 45.8+ MB


In [8]:
%%time
# Загружаем диалоги клиентов
dialog_df = pq.read_table(PATH_DATASET_OUTPUT + 'compress_targets_08_06_2024.parquet').to_pandas()
# targets_df = targets_df.rename(columns={'mon': 'report_next_end'})
targets_df = targets_df.reset_index()
targets_df = targets_df[['client_id', 'mon', 'target_1', 'target_2', 'target_3', 'target_4']]
targets_df.shape

In [9]:
%%time
# Загружаем факты продаж продуктов по трейн клиентам
targets_df = pq.read_table(PATH_DATASET_OUTPUT + 'compress_targets_08_06_2024.parquet').to_pandas()
# targets_df = targets_df.rename(columns={'mon': 'report_next_end'})
targets_df = targets_df.reset_index()
targets_df = targets_df[['client_id', 'mon', 'target_1', 'target_2', 'target_3', 'target_4']]
targets_df.shape

CPU times: total: 4 s
Wall time: 5.07 s


(11686066, 6)

In [10]:
# В данных встречаются дубли клиент+отчетный месяц. Там всегда нули, поэтому просто удаляем дубли 
targets_df = targets_df.drop_duplicates(subset=['client_id', 'mon'])
targets_df.shape

(11654375, 6)

In [11]:
%%time
# Рассчитываем факт приобретения клиентом когда-либо продукта 1 или 2/3/4
def get_group_targets(df:pd.DataFrame) -> pd.DataFrame:
    # Факт приобретения клиентом когда-либо продукта 1 или 2/3/4
    df['is_target'] = df[['target_1', 'target_2', 'target_3', 'target_4']].max(axis=1)
    
    # Расширеный факт приобретения клиентом когда-либо группы продуктов 
    df['is_target_1_2'] = df[['target_1', 'target_2']].max(axis=1)
    df['is_target_1_3'] = df[['target_1', 'target_3']].max(axis=1)
    df['is_target_1_4'] = df[['target_1', 'target_4']].max(axis=1)
    df['is_target_2_3'] = df[['target_2', 'target_3']].max(axis=1)
    df['is_target_2_4'] = df[['target_2', 'target_4']].max(axis=1)
    df['is_target_3_4'] = df[['target_3', 'target_4']].max(axis=1)

    df['is_target_123'] = df[['target_1', 'target_2', 'target_3']].max(axis=1)
    df['is_target_134'] = df[['target_1', 'target_3', 'target_4']].max(axis=1)
    df['is_target_124'] = df[['target_1', 'target_2', 'target_4']].max(axis=1)
    df['is_target_234'] = df[['target_2', 'target_3', 'target_4']].max(axis=1)
    
    # Второй расширеный факт приобретения клиентом когда-либо группы продуктов 
    df['is_target_1_and_2'] = np.where(df[['target_1', 'target_2']].sum(axis=1) == 2, 1,0)
    df['is_target_1_and_3'] = np.where(df[['target_1', 'target_3']].sum(axis=1) == 2, 1,0)
    df['is_target_1_and_4'] = np.where(df[['target_1', 'target_4']].sum(axis=1) == 2, 1,0)
    df['is_target_2_and_3'] = np.where(df[['target_2', 'target_3']].sum(axis=1) == 2, 1,0)
    df['is_target_2_and_4'] = np.where(df[['target_2', 'target_4']].sum(axis=1) == 2, 1,0)
    df['is_target_3_and_4'] = np.where(df[['target_3', 'target_4']].sum(axis=1) == 2, 1,0)
    
    df['is_target_and_123'] = np.where(df[['target_1', 'target_2', 'target_3']].sum(axis=1) == 2, 1,0)
    df['is_target_and_134'] = np.where(df[['target_1', 'target_3', 'target_4']].sum(axis=1) == 2, 1,0)
    df['is_target_and_124'] = np.where(df[['target_1', 'target_2', 'target_4']].sum(axis=1) == 2, 1,0)
    df['is_target_and_234'] = np.where(df[['target_2', 'target_3', 'target_4']].sum(axis=1) == 2, 1,0)    
    
    # кол-во купленных продуктов
    df['is_target_cnt'] = df[['target_1', 'target_2', 'target_3', 'target_4']].sum(axis=1)

    return df

targets_df = get_group_targets(targets_df)
targets_df.shape

CPU times: total: 18.4 s
Wall time: 22.8 s


(11654375, 28)

In [12]:
target_columns = ['target_1', 'target_2', 'target_3', 'target_4',
                  'is_target', 'is_target_1_2', 'is_target_1_3',
                  'is_target_1_4', 'is_target_2_3', 'is_target_2_4', 'is_target_3_4',
                  'is_target_1_and_2', 'is_target_1_and_3', 'is_target_1_and_4',
                  'is_target_2_and_3', 'is_target_2_and_4', 'is_target_3_and_4',
                  'is_target_123', 'is_target_134', 'is_target_124', 'is_target_234', 
                  'is_target_cnt']
len(target_columns)

22

In [13]:
%%time
mon_targets_df = targets_df.groupby(by='mon').agg(
    sum_target_1 = ('target_1', sum), 
    sum_target_2 = ('target_2', sum), 
    sum_target_3 = ('target_3', sum), 
    sum_target_4 = ('target_4', sum), 
    sum_is_target = ('is_target', sum), 
    sum_is_target_1_2 = ('is_target_1_2', sum), 
    sum_is_target_1_3 = ('is_target_1_3', sum), 
    sum_is_target_1_4 = ('is_target_1_4', sum), 
    sum_is_target_2_3 = ('is_target_2_3', sum), 
    sum_is_target_2_4 = ('is_target_2_4', sum), 
    sum_is_target_3_4 = ('is_target_3_4', sum), 
    sum_is_target_1_and_2 = ('is_target_1_and_2', sum), 
    sum_is_target_1_and_3 = ('is_target_1_and_3', sum), 
    sum_is_target_1_and_4 = ('is_target_1_and_4', sum), 
    sum_is_target_2_and_3 = ('is_target_2_and_3', sum), 
    sum_is_target_2_and_4 = ('is_target_2_and_4', sum), 
    sum_is_target_3_and_4 = ('is_target_3_and_4', sum), 
    sum_is_target_123 = ('is_target_123', sum), 
    sum_is_target_134 = ('is_target_134', sum), 
    sum_is_target_124 = ('is_target_124', sum), 
    sum_is_target_234 = ('is_target_234', sum), 
    sum_is_target_cnt = ('is_target_cnt', sum),  
)
mon_targets_df = mon_targets_df.reset_index()
mon_targets_df['next_mon'] = mon_targets_df['mon'].shift(1)
mon_targets_df['pre_mon'] = mon_targets_df['mon'].shift(-1)

mon_targets_df.shape

CPU times: total: 1.64 s
Wall time: 1.92 s


(12, 25)

In [ ]:
%%time
# Формируем фичи по таргету
begin_date = datetime(2022, 1, 1, 0, 0, 0)
start_date = datetime(2022, 1, 1, 0, 0, 0)

end_date = datetime(2023, 3, 31, 0, 0, 0)

# Бланк-датафрейм с клиентами 
uniq_clients_df = targets_df[['client_id']].drop_duplicates()
# Итоговый датасет 
union_client_agg_df = pd.DataFrame()

# Бежим по месяцам и расчитываем статистики для клиента берем предыдущие месяцы
for i in trange(((end_date - start_date).days//30 + 1)):
    end_date = start_date + relativedelta(months=1) - relativedelta(days=1)
    print(f'start: {start_date}, end: {end_date}')    
    select_mon_current_df = targets_df[targets_df['mon'].between(start_date, end_date)]
    select_mon_full_df = targets_df[targets_df['mon'].between(begin_date, end_date)]
    print(select_mon_current_df.shape, select_mon_full_df.shape)
    
    client_agg_df = uniq_clients_df.copy()
    report_next_end = start_date + relativedelta(months=2) - relativedelta(days=1)
    client_agg_df['report_next_end'] = report_next_end
    client_agg_df = client_agg_df.set_index('client_id')
    
    select_mon_full_df = select_mon_full_df.set_index('client_id')
    for cur_tar in ['target_1', 'target_2', 'target_3', 'target_4', 'is_target']:
        # Расчитываем даты первой и последней покупки продукта 
        min_max_date_buy = select_mon_full_df[select_mon_full_df[cur_tar] == 1].groupby(by='client_id').agg(
                                    first_day_buy = ('mon', min),
                                    last_day_buy = ('mon', max),
        )
        #break
        client_agg_df = client_agg_df.merge(min_max_date_buy, left_index=True, right_index=True, how='left')
        client_agg_df[f'days_first_buy_{cur_tar}'] = (client_agg_df['report_next_end'] - client_agg_df['first_day_buy']).dt.days
        client_agg_df[f'days_last_buy_{cur_tar}'] = (client_agg_df['report_next_end'] - client_agg_df['last_day_buy']).dt.days
        client_agg_df = client_agg_df.drop(columns=['first_day_buy', 'last_day_buy'])
        client_agg_df = client_agg_df.fillna(0)
    
    # Количество покупок продуктов за весь период
    client_agg_df = client_agg_df.merge(
            select_mon_full_df.groupby(by='client_id').agg(
                    sum_target_1_by_all_period = ('target_1', sum),
                    sum_target_2_by_all_period = ('target_2', sum),
                    sum_target_3_by_all_period = ('target_3', sum),
                    sum_target_4_by_all_period = ('target_4', sum),
                ), left_index=True, right_index=True, how='left'
            )
    
    # Доля покупок по продуктам
    client_agg_df['sum_all_target_by_all_period'] = client_agg_df[['sum_target_1_by_all_period', 'sum_target_2_by_all_period', 'sum_target_3_by_all_period', 'sum_target_4_by_all_period']].sum(axis=1)
    client_agg_df['prc_target_1by_all_trgs'] = (client_agg_df['sum_target_1_by_all_period'] / client_agg_df['sum_all_target_by_all_period']).fillna(0)
    client_agg_df['prc_target_2by_all_trgs'] = (client_agg_df['sum_target_2_by_all_period'] / client_agg_df['sum_all_target_by_all_period']).fillna(0)
    client_agg_df['prc_target_3by_all_trgs'] = (client_agg_df['sum_target_3_by_all_period'] / client_agg_df['sum_all_target_by_all_period']).fillna(0)
    client_agg_df['prc_target_4by_all_trgs'] = (client_agg_df['sum_target_4_by_all_period'] / client_agg_df['sum_all_target_by_all_period']).fillna(0)
    
    # Сколько в среднем в месяц клиент покупает продуктов 
    cnt_month = (end_date - begin_date).days / 30
    client_agg_df['mean_all_target_by_per_mon'] = client_agg_df['sum_all_target_by_all_period'] / cnt_month
    client_agg_df['mean_target_1_by_per_mon'] = client_agg_df['sum_target_1_by_all_period'] / cnt_month
    client_agg_df['mean_target_2_by_per_mon'] = client_agg_df['sum_target_2_by_all_period'] / cnt_month
    client_agg_df['mean_target_3_by_per_mon'] = client_agg_df['sum_target_3_by_all_period'] / cnt_month
    client_agg_df['mean_target_4_by_per_mon'] = client_agg_df['sum_target_4_by_all_period'] / cnt_month    
    
    # Количество покупок продуктов за 30 дней
    client_agg_df = client_agg_df.merge(
            select_mon_full_df[select_mon_full_df['mon'] >= report_next_end - relativedelta(months=1)].groupby(by='client_id').agg(
                    sum_target_1_by_1_mon = ('target_1', sum),
                    sum_target_2_by_1_mon = ('target_2', sum),
                    sum_target_3_by_1_mon = ('target_3', sum),
                    sum_target_4_by_1_mon = ('target_4', sum),
                ), left_index=True, right_index=True, how='left'
            )
    # Количество покупок продуктов за 60 дней
    client_agg_df = client_agg_df.merge(
            select_mon_full_df[select_mon_full_df['mon'] >= report_next_end - relativedelta(months=2)].groupby(by='client_id').agg(
                    sum_target_1_by_2_mon = ('target_1', sum),
                    sum_target_2_by_2_mon = ('target_2', sum),
                    sum_target_3_by_2_mon = ('target_3', sum),
                    sum_target_4_by_2_mon = ('target_4', sum),
                ), left_index=True, right_index=True, how='left'
            )
    
    # Количество покупок продуктов за 90 дней
    client_agg_df = client_agg_df.merge(
            select_mon_full_df[select_mon_full_df['mon'] >= report_next_end - relativedelta(months=3)].groupby(by='client_id').agg(
                    sum_target_1_by_3_mon = ('target_1', sum),
                    sum_target_2_by_3_mon = ('target_2', sum),
                    sum_target_3_by_3_mon = ('target_3', sum),
                    sum_target_4_by_3_mon = ('target_4', sum),
                ), left_index=True, right_index=True, how='left'
            )
    # Количество покупок продуктов за 120 дней
    client_agg_df = client_agg_df.merge(
            select_mon_full_df[select_mon_full_df['mon'] >= report_next_end - relativedelta(months=4)].groupby(by='client_id').agg(
                    sum_target_1_by_4_mon = ('target_1', sum),
                    sum_target_2_by_4_mon = ('target_2', sum),
                    sum_target_3_by_4_mon = ('target_3', sum),
                    sum_target_4_by_4_mon = ('target_4', sum),
                ), left_index=True, right_index=True, how='left'
            )
    # Количество покупок продуктов за 150 дней
    client_agg_df = client_agg_df.merge(
            select_mon_full_df[select_mon_full_df['mon'] >= report_next_end - relativedelta(months=5)].groupby(by='client_id').agg(
                    sum_target_1_by_5_mon = ('target_1', sum),
                    sum_target_2_by_5_mon = ('target_2', sum),
                    sum_target_3_by_5_mon = ('target_3', sum),
                    sum_target_4_by_5_mon = ('target_4', sum),
                ), left_index=True, right_index=True, how='left'
            )
    # Количество покупок продуктов за 180 дней
    client_agg_df = client_agg_df.merge(
            select_mon_full_df[select_mon_full_df['mon'] >= report_next_end - relativedelta(months=6)].groupby(by='client_id').agg(
                    sum_target_1_by_6_mon = ('target_1', sum),
                    sum_target_2_by_6_mon = ('target_2', sum),
                    sum_target_3_by_6_mon = ('target_3', sum),
                    sum_target_4_by_6_mon = ('target_4', sum),
                ), left_index=True, right_index=True, how='left'
            )
    
    # Период неактивности    
    period_noactive_target = select_mon_full_df[select_mon_full_df['target_1'] == 0].sort_values(by=['client_id', 'mon'])
    period_noactive_target['shift_mon'] = period_noactive_target.groupby('client_id')['mon'].shift(1)
    period_noactive_target['period_noactive_target'] = (period_noactive_target['mon'] - period_noactive_target['shift_mon']).dt.days.fillna(0)
    period_noactive_target = period_noactive_target.groupby(by='client_id').agg(
           max_period_noactive_target = ('period_noactive_target', max),
           min_period_noactive_target = ('period_noactive_target', min),
           avg_period_noactive_target = ('period_noactive_target', np.mean),
           median_period_noactive_target = ('period_noactive_target', np.median),
    )
    client_agg_df = client_agg_df.merge(period_noactive_target, left_index=True, right_index=True, how='left')
    
    union_client_agg_df = pd.concat([union_client_agg_df, client_agg_df])
    start_date = start_date + relativedelta(months=1)
union_client_agg_df.shape

  0%|          | 0/16 [00:00<?, ?it/s]

start: 2022-01-01 00:00:00, end: 2022-01-31 00:00:00
(0, 28) (0, 28)


  6%|▋         | 1/16 [00:06<01:33,  6.22s/it]

start: 2022-02-01 00:00:00, end: 2022-02-28 00:00:00
(994380, 28) (994380, 28)


 12%|█▎        | 2/16 [00:34<04:32, 19.48s/it]

...

start: 2023-01-01 00:00:00, end: 2023-01-31 00:00:00
(853892, 28) (11654375, 28)


 81%|████████▏ | 13/16 [07:45<02:02, 40.69s/it]

start: 2023-02-01 00:00:00, end: 2023-02-28 00:00:00
(0, 28) (11654375, 28)


 88%|████████▊ | 14/16 [08:20<01:18, 39.10s/it]

start: 2023-03-01 00:00:00, end: 2023-03-31 00:00:00
(0, 28) (11654375, 28)


In [ ]:
%%time
# Уменьшение размера датафрейма, для таргетов, транзакцй и для фичей
def series_to_int(col_df:pd.Series):
    """
    Перевод в целочисленные типы
    """
    min_val = col_df.min()
    max_val = col_df.max()
    if min_val >= -128 and max_val <= 127:
        col_df = col_df.astype('int8')
    elif min_val >= -32768 and max_val <= 32767:
        col_df = col_df.astype('int16')
    elif min_val >= -2147483648 and max_val <= 2147483647:
        col_df = col_df.astype('int32')
    else:
        col_df = col_df.astype('int64')
    return col_df

def compression_df(df:pd.DataFrame(), datetime_cols:List[str]=[], category_cols:List[str]=[]):
    """
    Уменьшение размера датафрейма, для таргетов, транзакцй и для фичей
    """
    float64_cols = list(df.select_dtypes(include='float64'))  
    df[float64_cols] = df[float64_cols].astype('float32')
    for col in df.columns:
        if col in category_cols:
            df[col] = df[col].astype('category')
        elif col in datetime_cols:
            if df[col].dtypes == 'object':
                df[col] = pd.to_datetime(df[col])
        # Если колонка содержит числа 
        elif is_integer_dtype(df[col]):
            if df[col].dtypes == 'int8':
                continue
            else:
                df[col] = series_to_int(df[col])
        elif is_float_dtype(df[col]):
            # Возможно ли перевести в число
            if np.array_equal(df[col].fillna(0), df[col].fillna(0).astype(int)):
                df[col] = df[col].fillna(0)
                df[col] = series_to_int(df[col])
    return df
union_client_agg_df = compression_df(union_client_agg_df, 
                            datetime_cols=['report_end' ,'report_next_end'],
                           )

mon_targets_df = compression_df(mon_targets_df, 
                            datetime_cols=['mon', 'pre_mon', 'next_mon'],
                           )
union_client_agg_df.shape, mon_targets_df.shape

## Объединяем с агрегированными данными по месяцам

In [16]:
# union_client_agg_df['report_next_end'].min()
union_client_agg_df = union_client_agg_df.reset_index('client_id').set_index('report_next_end')
union_client_agg_df.shape

(15910080, 53)

In [17]:
%%time
union_client_agg_df = union_client_agg_df.merge(
                mon_targets_df.drop(columns=['mon', 'next_mon']).set_index('pre_mon').add_prefix('agg_premon_'),
                left_index=True,
                right_index=True,    
                how='left')
gc.collect()

union_client_agg_df = union_client_agg_df.merge(
                mon_targets_df.drop(columns=['pre_mon', 'next_mon']).set_index('mon').add_prefix('agg_curmon_'),
                left_index=True,
                right_index=True,    
                how='left')

gc.collect()

union_client_agg_df = union_client_agg_df.merge(
                mon_targets_df.drop(columns=['pre_mon', 'mon']).set_index('next_mon').add_prefix('agg_nxtmon_'),
                left_index=True,
                right_index=True,    
                how='left')

gc.collect()
union_client_agg_df = union_client_agg_df.fillna(0)
# union_client_agg_df = union_client_agg_df.reset_index().rename(columns={'index': 'report_next_end'}).set_index(['client_id','report_next_end'])
union_client_agg_df = union_client_agg_df.reset_index().rename(columns={'index': 'report_next_end'})
union_client_agg_df = union_client_agg_df.sort_values(by=['client_id','report_next_end']).set_index(['client_id','report_next_end'])

union_client_agg_df.shape

CPU times: total: 1min 7s
Wall time: 1min 3s


(15910080, 118)

In [18]:
%%time
# Сохраняем в файл оптимизированный файл 
union_client_agg_df.to_parquet(PATH_DATASET_OUTPUT + 'client_agg_target_09_06_2024.parquet')

CPU times: total: 35.3 s
Wall time: 31.7 s
